# Первичный анализ ряда

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error

# --- Настройки ---
# Укажи путь к твоему файлу
file_path = 'CPI.xlsx'
# Названия колонок в твоем файле
date_column = 'Дата'
cpi_column = 'ИПЦ'
# Определяем период для теста (например, последние 24 месяца)
test_months = 24
# --- Конец Настроек ---

# Чтение данных
df = pd.read_excel(file_path, engine='openpyxl', sheet_name='ИПЦ')

# Преобразование колонки с датой и установка ее как индекс
df[date_column] = pd.to_datetime(df[date_column])
df = df.set_index(date_column)

# Выбор нужной колонки с ИПЦ
cpi_data = df[[cpi_column]]

# Удаление пропусков, если они есть (простой вариант)
cpi_data = cpi_data.dropna()

# Определение дат для разделения на train и test
# data_do - последняя дата в данных
data_do = cpi_data.index.max()
# train_do - дата начала тестового периода
# Отнимаем нужное количество месяцев от конца
train_do_date = data_do - pd.DateOffset(months=test_months - 1)
train_do = train_do_date.strftime('%Y-%m-%d') # Формат как в исходном коде

print(f"Данные используются до: {data_do.strftime('%Y-%m-%d')}")
print(f"Тестовый период начинается с: {train_do}")

# Разделение данных
train = cpi_data[cpi_data.index < train_do]
test = cpi_data[cpi_data.index >= train_do]

print(f"Размер обучающей выборки: {len(train)}")
print(f"Размер тестовой выборки: {len(test)}")

# Проверим, что тестовая выборка имеет нужную длину
if len(test) != test_months:
    print(f"Внимание: Длина тестовой выборки ({len(test)}) не равна {test_months} месяцев.")
    # Можно скорректировать train_do, если нужно точно test_months точек
    train_do_date = cpi_data.index[-test_months]
    train_do = train_do_date.strftime('%Y-%m-%d')
    train = cpi_data[cpi_data.index < train_do]
    test = cpi_data[cpi_data.index >= train_do]
    print(f"Скорректированный train_do: {train_do}, новая длина теста: {len(test)}")


# Выбираем только значения ИПЦ для обучения
train_values = train[cpi_column]
test_values = test[cpi_column]


plot_start_date = '2018-01-01'

In [ ]:
random_seed = 42

In [ ]:
# --- Настройки (если нужно поменять) ---
cpi_value_column = 'ИПЦ' # Убедись, что это имя твоей колонки
# --- Конец Настроек ---

# Используем переменную cpi_data из предыдущих шагов
inflation_series = cpi_data[cpi_value_column]

# Построение графика
plt.figure(figsize=(14, 7))
plt.plot(inflation_series.index, inflation_series, label=f'{cpi_value_column}')
plt.title('Темпы прироста потребительских цен, в процентах к предыдущему месяцу, сезонность устранена')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller

# --- Настройки ---
cpi_value_column = 'ИПЦ'
significance_level = 0.05 # Уровень значимости
# --- Конец Настроек ---

# Используем переменную cpi_data из предыдущих шагов
inflation_series = train[cpi_value_column]

print("--- Результаты теста Дики-Фуллера (ADF) ---")
# Проводим тест
# autolag='AIC' позволяет тесту автоматически подобрать оптимальное число лагов
adf_test_result = adfuller(inflation_series, autolag='AIC')

# Извлекаем результаты
adf_statistic = adf_test_result[0]
p_value = adf_test_result[1]
lags_used = adf_test_result[2]
critical_values = adf_test_result[4]

# Выводим результаты
print(f'ADF Statistic: {adf_statistic:.4f}')
print(f'p-value: {p_value:.4f}')
print(f'Число использованных лагов: {lags_used}')
print('Критические значения:')
for key, value in critical_values.items():
    print(f'\t{key}: {value:.4f}')

# Интерпретация результата
print("\nИнтерпретация:")
if p_value <= significance_level:
    print(f"p-value ({p_value:.4f}) меньше уровня значимости ({significance_level}).")
    print("Отвергаем нулевую гипотезу (H0). Ряд, скорее всего, СТАЦИОНАРЕН.")
else:
    print(f"p-value ({p_value:.4f}) больше уровня значимости ({significance_level}).")
    print("Не можем отвергнуть нулевую гипотезу (H0). Ряд, скорее всего, НЕ СТАЦИОНАРЕН.")
print("---------------------------------------------")

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

# --- Настройки ---
cpi_value_column = 'ИПЦ'
lags_to_plot = 40 # Сколько лагов отображать на графике
# --- Конец Настроек ---

# Используем переменную cpi_data из предыдущих шагов
inflation_series = train[cpi_value_column]

# Построение графика ACF
fig, ax = plt.subplots(figsize=(12, 6))
plot_acf(inflation_series, lags=lags_to_plot, ax=ax, title='Автокорреляционная функция (ACF)')
ax.grid(True)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

# --- Настройки ---
cpi_value_column = 'ИПЦ'
lags_to_plot = 40 # Сколько лагов отображать на графике
# --- Конец Настроек ---

# Используем переменную cpi_data из предыдущих шагов
inflation_series = train[cpi_value_column]

# Построение графика PACF
fig, ax = plt.subplots(figsize=(12, 6))
# method='ywm' (Yule-Walker Mle) - стандартный метод расчета
plot_pacf(inflation_series, lags=lags_to_plot, method='ywm', ax=ax, title='Частичная автокорреляционная функция (PACF)')
ax.grid(True)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

# --- Настройки ---
cpi_value_column = 'ИПЦ'
lags_to_plot = 40  # Сколько лагов отображать на графике
# --- Конец Настроек ---

# Используем переменную train из предыдущих шагов
inflation_series = train[cpi_value_column]

# Создаем фигуру с двумя подграфиками (вертикально)
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))

# Построение ACF на первом подграфике
plot_acf(inflation_series, lags=lags_to_plot, ax=ax1, title='Автокорреляционная функция (ACF)')
ax1.grid(True)

# Построение PACF на втором подграфике
plot_pacf(inflation_series, lags=lags_to_plot, method='ywm', ax=ax2, title='Частичная автокорреляционная функция (PACF)')
ax2.grid(True)

# Автоматическая регулировка отступов между графиками
plt.tight_layout()
plt.show()

* Визуальный анализ графика инфляции и результаты теста Дики-Фуллера (p-value << 0.05) указывают на стационарность временного ряда.
* Графики автокорреляционной (ACF) и частичной автокорреляционной (PACF) функций показывают характерную для авторегрессионного процесса первого порядка (AR(1)) структуру: ACF постепенно затухает, а PACF резко обрывается после первого лага.
* Следовательно, адекватной моделью для данного ряда может быть AR(1).

# ARIMA и простые бенчмарки

In [ ]:
result_rmse = pd.DataFrame(columns=['method', 'rmse'])
df_horizon_rmse = pd.DataFrame(columns=['method', 'horizon', 'rmse'])
recursive_forecasts = {}
horizons_to_evaluate = [1, 2, 3, 6, 12, 18, 24]

In [ ]:
# --- БЛОК 3: Простые Бенчмарки + Инициализация Результатов ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# --- ПРЕДПОЛАГАЕМ, ЧТО ПЕРЕМЕННЫЕ СУЩЕСТВУЮТ ИЗ ПРЕДЫДУЩИХ БЛОКОВ ---
# train_values, test_values: pd.Series с обучающими и тестовыми значениями ИПЦ
# train, test: pd.DataFrames с обучающими и тестовыми данными (для индексов и графиков)
# test_months: Количество месяцев в тесте (24)
# horizons_to_evaluate: Список горизонтов [1, 2, 3, 6, 12, 18, 24]
# plot_start_date: Дата начала для графиков прогнозов
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

print("\n--- Инициализация структур для результатов ---")
result_rmse = pd.DataFrame(columns=['method', 'rmse'])
df_horizon_rmse = pd.DataFrame(columns=['method', 'horizon', 'rmse'])
recursive_forecasts = {} # Словарь для ВСЕХ финальных прогнозов

print("Структуры result_rmse, df_horizon_rmse, recursive_forecasts инициализированы.")

print("\n--- Генерация и Оценка Простых Бенчмарков ---")

# --- Модель Mean ---
model_name_mean = "Mean"
mean_forecast_value = train_values.mean()
forecast_mean = np.full(test_months, mean_forecast_value)
recursive_forecasts[model_name_mean] = forecast_mean
print(f"\n{model_name_mean}: Прогноз = {mean_forecast_value:.4f}")
overall_rmse_mean = np.sqrt(mean_squared_error(test_values, forecast_mean))
print(f"Общий RMSE: {overall_rmse_mean:.4f}")
result_rmse = pd.concat([result_rmse, pd.DataFrame([{'method': model_name_mean, 'rmse': overall_rmse_mean}])], ignore_index=True)
horizon_results_list_mean = []
print("RMSE по горизонтам:")
for h in horizons_to_evaluate:
    rmse_h = np.sqrt(mean_squared_error(test_values[:h], forecast_mean[:h]))
    print(f"  1-{h} мес.: {rmse_h:.4f}")
    horizon_results_list_mean.append({'method': model_name_mean, 'horizon': h, 'rmse': rmse_h})
df_horizon_rmse = pd.concat([df_horizon_rmse, pd.DataFrame(horizon_results_list_mean)], ignore_index=True)

# --- Модель Median ---
model_name_median = "Median"
median_forecast_value = train_values.median()
forecast_median = np.full(test_months, median_forecast_value)
recursive_forecasts[model_name_median] = forecast_median
print(f"\n{model_name_median}: Прогноз = {median_forecast_value:.4f}")
overall_rmse_median = np.sqrt(mean_squared_error(test_values, forecast_median))
print(f"Общий RMSE: {overall_rmse_median:.4f}")
result_rmse = pd.concat([result_rmse, pd.DataFrame([{'method': model_name_median, 'rmse': overall_rmse_median}])], ignore_index=True)
horizon_results_list_median = []
print("RMSE по горизонтам:")
for h in horizons_to_evaluate:
    rmse_h = np.sqrt(mean_squared_error(test_values[:h], forecast_median[:h]))
    print(f"  1-{h} мес.: {rmse_h:.4f}")
    horizon_results_list_median.append({'method': model_name_median, 'horizon': h, 'rmse': rmse_h})
df_horizon_rmse = pd.concat([df_horizon_rmse, pd.DataFrame(horizon_results_list_median)], ignore_index=True)

# --- Модель Trend ---
model_name_trend = "Trend"
print(f"\n{model_name_trend}:")
try:
    X_train_trend = np.arange(len(train_values)).reshape(-1, 1)
    trend_model = LinearRegression().fit(X_train_trend, train_values)
    X_test_trend = np.arange(len(train_values), len(train_values) + test_months).reshape(-1, 1)
    forecast_trend = trend_model.predict(X_test_trend)
    recursive_forecasts[model_name_trend] = forecast_trend
    print("Прогноз сгенерирован.")
    overall_rmse_trend = np.sqrt(mean_squared_error(test_values, forecast_trend))
    print(f"Общий RMSE: {overall_rmse_trend:.4f}")
    result_rmse = pd.concat([result_rmse, pd.DataFrame([{'method': model_name_trend, 'rmse': overall_rmse_trend}])], ignore_index=True)
    horizon_results_list_trend = []
    print("RMSE по горизонтам:")
    for h in horizons_to_evaluate:
        rmse_h = np.sqrt(mean_squared_error(test_values[:h], forecast_trend[:h]))
        print(f"  1-{h} мес.: {rmse_h:.4f}")
        horizon_results_list_trend.append({'method': model_name_trend, 'horizon': h, 'rmse': rmse_h})
    df_horizon_rmse = pd.concat([df_horizon_rmse, pd.DataFrame(horizon_results_list_trend)], ignore_index=True)
except Exception as e:
    print(f"Ошибка при генерации/оценке прогноза Trend: {e}")
    recursive_forecasts[model_name_trend] = np.zeros(test_months) * np.nan

print("\n--- Простые бенчмарки рассчитаны и добавлены в таблицы ---")

print("\nТекущая таблица общих RMSE:")
print(result_rmse.sort_values(by='rmse'))
print("\nТекущая таблица RMSE по горизонтам:")
pivot_table_simple = df_horizon_rmse.pivot(index='horizon', columns='method', values='rmse')
# Сортируем колонки по текущему общему RMSE
current_simple_order = result_rmse['method'].tolist()
print(pivot_table_simple[current_simple_order].round(4))

In [ ]:
# --- БЛОК 3А: Визуализация Простых Бенчмарков ---

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# --- ПРЕДПОЛАГАЕМ, ЧТО ПЕРЕМЕННЫЕ СУЩЕСТВУЮТ ИЗ БЛОКА 3 ---
# train_values, test_values: pd.Series
# train, test: pd.DataFrames
# forecast_mean, forecast_median, forecast_trend: numpy arrays
# df_horizon_rmse: DataFrame с результатами по горизонтам для Mean, Median, Trend
# horizons_to_evaluate: Список горизонтов
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

print("\n--- Визуализация Простых Бенчмарков ---")

# 1. График прогнозов на всем периоде
plt.figure(figsize=(15, 7))
# Рисуем весь реальный ряд (train + test)
plt.plot(pd.concat([train_values[train_values.index >= plot_start_date], test_values]).index,
         pd.concat([train_values[train_values.index >= plot_start_date], test_values]),
         label='Реальные данные (Весь ряд)', color='grey', alpha=0.7)

# Создаем полные ряды прогнозов для графика
full_index = pd.concat([train_values, test_values]).index
mean_full_forecast = pd.Series(np.nan, index=full_index)
mean_full_forecast.loc[test.index] = forecast_mean # Заполняем только тестовый период

median_full_forecast = pd.Series(np.nan, index=full_index)
median_full_forecast.loc[test.index] = forecast_median

trend_full_forecast = pd.Series(np.nan, index=full_index)
trend_full_forecast.loc[test.index] = forecast_trend
# Добавим линию тренда и на обучающий период для наглядности
trend_train_pred = trend_model.predict(np.arange(len(train_values)).reshape(-1, 1))
trend_full_forecast.loc[train.index] = trend_train_pred

mean_full_forecast = mean_full_forecast[mean_full_forecast.index >= plot_start_date]
median_full_forecast = median_full_forecast[median_full_forecast.index >= plot_start_date]
trend_full_forecast = trend_full_forecast[trend_full_forecast.index >= plot_start_date]

# Рисуем прогнозы
plt.plot(mean_full_forecast.index, mean_full_forecast,
         label=f'Прогноз Mean ({mean_forecast_value:.2f})', color='blue', linestyle='--')
plt.plot(median_full_forecast.index, median_full_forecast,
         label=f'Прогноз Median ({median_forecast_value:.2f})', color='black', linestyle='--')
plt.plot(trend_full_forecast.index, trend_full_forecast,
         label='Прогноз Trend', color='red', linestyle='--')


plt.title('Прогнозы простых бенчмарков на тестовом периоде')
plt.xlabel('Дата')
plt.ylabel('ИПЦ (месячный рост)')
# Ограничим ось Y для лучшей видимости прогнозов (можно закомментировать)
# plt.ylim(min(test_values.min(), forecast_trend.min()) - 0.5, test_values.max() + 0.5)
plt.axvline(train_values.index[-1], color='gray', linestyle=':', label='Начало теста') # Вертикальная линия
plt.legend()
plt.grid(True)
plt.show()


# 2. График RMSE vs Горизонт для простых бенчмарков
plt.figure(figsize=(10, 5))
simple_methods = ["Mean", "Median", "Trend"]
simple_data_to_plot = df_horizon_rmse[df_horizon_rmse['method'].isin(simple_methods)]

# Используем seaborn для разделения по 'method'
sns.lineplot(data=simple_data_to_plot, x='horizon', y='rmse', hue='method',
             marker='o', palette=['blue', 'black', 'red']) # Цвета как на графике выше

plt.title('Зависимость RMSE от горизонта прогноза для простых бенчмарков')
plt.xlabel('Горизонт прогноза (месяцы)')
plt.ylabel('RMSE')
plt.xticks(horizons_to_evaluate)
plt.grid(True)
plt.legend(title='Модель')
plt.show()

In [ ]:
# --- БЛОК 4: Модель ARIMA ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error

# --- ПРЕДПОЛАГАЕМ, ЧТО ПЕРЕМЕННЫЕ СУЩЕСТВУЮТ ИЗ ПРЕДЫДУЩИХ БЛОКОВ ---
# train_values, test_values: pd.Series
# train, test: pd.DataFrames
# test_months, horizons_to_evaluate, plot_start_date
# result_rmse, df_horizon_rmse, recursive_forecasts
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

model_name_arima = "ARIMA"

print(f"\n--- Обучение и Прогнозирование: {model_name_arima} ---")

# 1. Подбор и обучение модели
stepwise_model_arima = auto_arima(train_values,
                                  start_p=1, start_q=1, max_p=5, max_q=5,
                                  m=1, seasonal=False, d=0,
                                  test='adf', trace=False,
                                  error_action='ignore', suppress_warnings=True,
                                  stepwise=True, information_criterion='aic')
print(f"\nВыбранная модель: {stepwise_model_arima.order} {stepwise_model_arima.seasonal_order}")

# 2. Генерация рекурсивного прогноза
forecast_arima = stepwise_model_arima.predict(n_periods=test_months)
recursive_forecasts[model_name_arima] = forecast_arima[:test_months]
print(f"Прогноз {model_name_arima} сгенерирован и добавлен в словарь.")

# 3. Оценка RMSE
overall_rmse_arima = np.sqrt(mean_squared_error(test_values, forecast_arima))
print(f"\nОбщий RMSE на тесте для {model_name_arima}: {overall_rmse_arima:.4f}")

# Добавляем/Обновляем общий RMSE
result_rmse = result_rmse[result_rmse['method'] != model_name_arima]
result_arima_entry = {'method': model_name_arima, 'rmse': overall_rmse_arima}
result_rmse = pd.concat([result_rmse, pd.DataFrame([result_arima_entry])], ignore_index=True)

# Расчет и добавление RMSE по горизонтам
print("RMSE по горизонтам:")
df_horizon_rmse = df_horizon_rmse[df_horizon_rmse['method'] != model_name_arima]
horizon_results_list_arima = []
for h in horizons_to_evaluate:
    if h <= len(test_values):
        forecast_h = forecast_arima[:h]
        actual_h = test_values[:h]
        rmse_h = np.sqrt(mean_squared_error(actual_h, forecast_h))
        print(f"  1-{h} мес.: {rmse_h:.4f}") # Вывод промежуточного RMSE
        horizon_results_list_arima.append({'method': model_name_arima, 'horizon': h, 'rmse': rmse_h})
df_horizon_rmse_arima = pd.DataFrame(horizon_results_list_arima)
df_horizon_rmse = pd.concat([df_horizon_rmse, df_horizon_rmse_arima], ignore_index=True)
print("Результаты RMSE по горизонтам добавлены/обновлены.")

# 4. График прогноза
plt.figure(figsize=(12, 6))
history_to_plot = train_values[train_values.index >= plot_start_date]
plt.plot(history_to_plot.index, history_to_plot, label='Обучающая выборка (часть)')
plt.plot(test.index, test_values, label='Тестовая выборка (реальные)', color='orange', linewidth=2)
forecast_arima_series = pd.Series(forecast_arima, index=test.index[:len(forecast_arima)])
plt.plot(forecast_arima_series.index, forecast_arima_series, label=f'Прогноз {model_name_arima}', color='blue', linestyle='--')
plt.title(f'Прогноз ИПЦ с помощью {model_name_arima}')
plt.xlabel('Дата'); plt.ylabel('ИПЦ (месячный рост)'); plt.legend(); plt.grid(True)
plt.show()

# 5. График RMSE vs Горизонт (только для этой модели)
plt.figure(figsize=(10, 5))
plt.plot(df_horizon_rmse_arima['horizon'], df_horizon_rmse_arima['rmse'], marker='o', linestyle='-')
plt.title(f'Зависимость RMSE от горизонта прогноза для {model_name_arima}')
plt.xlabel('Горизонт прогноза (месяцы)'); plt.ylabel('RMSE')
plt.xticks(horizons_to_evaluate); plt.grid(True); plt.show()

print(f"--- Анализ {model_name_arima} завершен ---")

print("\nТекущая таблица общих RMSE:")
print(result_rmse.sort_values(by='rmse'))
print("\nТекущая таблица RMSE по горизонтам:")
pivot_table_current = df_horizon_rmse.pivot(index='horizon', columns='method', values='rmse')
current_order = result_rmse['method'].tolist() # Порядок по текущему общему RMSE
# Добавляем и сортируем колонки
for method in current_order:
     if method not in pivot_table_current.columns: pivot_table_current[method] = np.nan
print(pivot_table_current[current_order].round(4))

In [ ]:
print(stepwise_model_arima.summary())


* Применение автоматического подбора с помощью `auto_arima` (при `d=0`) выбрало модель **ARIMA(1,0,0)** как показавшую наименьшую ошибку RMSE (0.2627) на тестовой выборке. Этот выбор **согласуется** с выводами ручного анализа ряда, который подтвердил **стационарность** (тест ADF) и выявил **структуру AR(1)** (анализ ACF/PACF).

* Диагностика остатков модели выявила **отсутствие в них автокорреляции** (тест Льюнга-Бокса, Prob(Q) >> 0.05). Это **ключевой положительный результат**, поскольку он означает, что модель успешно извлекла из данных предсказуемую линейную структуру, и оставшийся "шум" не содержит явных паттернов, которые можно было бы использовать для улучшения *линейного* прогноза.

* В то же время, тесты остатков показали **статистически значимые отклонения от нормального распределения** (тест Харке-Бера, Prob(JB) << 0.05) и наличие **гетероскедастичности** (Prob(H) << 0.05). Хотя эти нарушения классических предпосылок важны для точности доверительных интервалов и статистической значимости *коэффициентов* модели, они **менее критичны для задачи точечного прогнозирования**, особенно при сравнении моделей по метрикам типа RMSE. Отсутствие автокорреляции в остатках позволяет считать линейную часть модели адекватной.

* Анализ ошибки прогноза на тестовой выборке показал немонотонную зависимость RMSE от горизонта: ошибка была выше на коротких горизонтах (1-3 мес., RMSE > 0.32), достигала минимума на горизонте 12 месяцев (RMSE ≈ 0.259) и незначительно возрастала далее, составляя 0.263 на полном 24-месячном горизонте.

* Учитывая лучший общий RMSE (0.263) среди рассмотренных ARIMA-вариантов и отсутствие автокорреляции остатков, модель ARIMA(1,0,0) принимается как обоснованный и сильный бенчмарк для дальнейшего сравнения с моделями машинного обучения. Анализ по горизонтам будет важен для детального сопоставления производительности разных методов.

# Feature Engineering


**Генерация признаков для моделей машинного обучения**

**Цель:**  
Создать набор информативных признаков для ML-моделей, используя **только историю самого ряда инфляции ('t')**. Это обеспечит корректность сравнения с моделью ARIMA, которая также использует только историю ряда.

**Выбор признаков**

Основан на подходе из статьи-источника и распространённых практиках анализа временных рядов:

1. **Лаги ряда (`t-1`, `t-6`, `t-12`)**  
   - Предполагается, что прошлые значения инфляции (особенно за предыдущий месяц, полгода и год) влияют на текущее значение.  
   - Лаг `t-1` особенно важен для рядов с AR(1)-структурой.

2. **Скользящие статистики** (среднее и стандартное отклонение за 3, 6, 12 месяцев)  
   - Помогают уловить:  
     - Локальный уровень инфляции (`t_mean_lag`),  
     - Волатильность (`t_std_lag`) за периоды, соответствующие кварталу, полугодию и году.  
   - Рассчитываются со сдвигом на 1 период назад (`.shift(1)`), чтобы избежать утечки данных.

3. **Порядковый номер месяца (`month`)**  
   - Добавляется для учёта возможных остаточных календарных эффектов, не уловленных другими признаками.

**Ограничение набора признаков**

Сознательно **не включаются все возможные лаги и окна**, чтобы:  
- Избежать "проклятия размерности" и переобучения на доступном объёме данных (~250 наблюдений),  
- Сохранить интерпретируемость модели,  
- Следовать методологии статьи-источника для сопоставимости.


In [ ]:
# --- Настройки ---
cpi_value_column = 'ИПЦ' # Имя столбца с инфляцией

lags_to_create = [1, 2, 3, 6, 12]

rolling_windows = [3, 6, 12]
# --- Конец Настроек ---


data_ml = cpi_data[[cpi_value_column]].copy()
data_ml = data_ml.rename(columns={cpi_value_column: 't'})

print("Исходные данные для ML:")
data_ml

In [ ]:
print("\n--- Создание лаговых признаков ---")
for lag in lags_to_create:
    data_ml[f't-{lag}'] = data_ml['t'].shift(lag)
    print(f"Создан признак t-{lag}")

print("\nDataFrame с лагами (начало):")
print(data_ml.head(max(lags_to_create) + 2))

In [ ]:
print("\n--- Создание скользящих статистик ---")

# Используем исходный ряд 't' для расчета
target_series = data_ml['t']

for window in rolling_windows:
    # Скользящее среднее
    rolling_mean = target_series.rolling(window=window).mean()
    data_ml[f't_mean_lag{window}'] = rolling_mean.shift(1) # Применяем shift(1)
    print(f"Создан признак t_mean_lag{window}")

    # Скользящее стандартное отклонение
    rolling_std = target_series.rolling(window=window).std()
    data_ml[f't_std_lag{window}'] = rolling_std.shift(1) # Применяем shift(1)
    print(f"Создан признак t_std_lag{window}")

print("\nDataFrame со статистиками (начало):")
# Покажем чуть больше строк, чтобы увидеть, как заполняются статистики
print(data_ml.head(max(rolling_windows) + max(lags_to_create) + 2)) # Примерно

**Кодирование признака "Месяц"**

Для учета возможных сезонных или календарных эффектов, которые могут оставаться в данных, в модель добавляется информация о месяце. Вместо простого порядкового номера (1-12), который вносит искусственную линейную зависимость и некорректно отражает близость декабря к январю, используется **циклическое кодирование**.

Месяц представляется двумя новыми признаками с помощью синуса и косинуса:

*   `month_sin = sin(2 * pi * month / 12)`
*   `month_cos = cos(2 * pi * month / 12)`

**Преимущества такого подхода:**

1.  **Отражение цикличности:** Эта пара признаков уникально идентифицирует каждый месяц и правильно передает модели информацию о том, что 12-й месяц близок к 1-му (их значения sin/cos близки).
2.  **Непрерывность:** Признаки являются непрерывными, что удобно для большинства ML алгоритмов.
3.  **Отсутствие искусственного порядка:** Модель не будет ошибочно считать, что декабрь "больше" января.

**Интерпретация:** Модель будет использовать эти два признака (`month_sin`, `month_cos`) для выявления любых паттернов, связанных с месяцем года. Например, определенная комбинация высоких значений `month_sin` и низких `month_cos` может соответствовать летним месяцам и быть связана с определенным уровнем инфляции, отличающимся от зимних месяцев (где комбинация sin/cos будет другой). Интерпретация влияния конкретного месяца становится менее прямой, чем с дамми-переменными, но модель получает более адекватное представление о времени года.

In [ ]:
print("\n--- Добавление циклических признаков месяца ---")
# Получаем номер месяца (1-12)
month_num = data_ml.index.month
# Вычисляем синус и косинус компоненты
data_ml['month_sin'] = np.sin(2 * np.pi * month_num / 12)
data_ml['month_cos'] = np.cos(2 * np.pi * month_num / 12)

print("Созданы признаки 'month_sin' и 'month_cos'")

data_ml[['t', 'month_sin', 'month_cos']] # Посмотрим на новые признаки

In [ ]:
print("\n--- Удаление строк с NaN ---")
initial_rows = len(data_ml)
data_ml.dropna(inplace=True)
final_rows = len(data_ml)
print(f"Удалено {initial_rows - final_rows} строк с NaN.")
print(f"Итоговый размер DataFrame для ML: {data_ml.shape}")
print("\nТипы данных:")
print(data_ml.info())

In [ ]:
data_ml

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit

# --- Настройки (из предыдущих шагов) ---
# Убедись, что train_do содержит правильную дату начала тестового периода
# train_do = '2023-02-01' # Пример, возьми свою дату
target_column = 't'
# Количество сплитов для TimeSeriesSplit (как в статье)
n_cv_splits = 10
# --- Конец Настроек ---

# 1. Разделение данных на train и test
print(f"--- Разделение данных (дата среза: {train_do}) ---")
train_ml = data_ml[data_ml.index < train_do]
test_ml = data_ml[data_ml.index >= train_do]

print(f"Размер обучающей выборки ML: {train_ml.shape}")
print(f"Размер тестовой выборки ML: {test_ml.shape}")

# Проверка, что размер тестовой выборки совпадает с ожидаемым (test_months)
if len(test_ml) != test_months:
     print(f"Предупреждение: Фактическая длина тестовой ML выборки ({len(test_ml)}) отличается от test_months ({test_months})")



In [ ]:
# 2. Определение X и y
print("\n--- Определение X (признаки) и y (цель) ---")
X_train = train_ml.drop(target_column, axis=1)
y_train = train_ml[target_column]

X_test = test_ml.drop(target_column, axis=1)
y_test = test_ml[target_column] # Это наши реальные значения для сравнения прогнозов

print("Размеры X_train:", X_train.shape)
print("Размеры y_train:", y_train.shape)
print("Размеры X_test:", X_test.shape)
print("Размеры y_test:", y_test.shape)

In [ ]:
# 3. Масштабирование признаков (для линейных моделей и LSTM)
print("\n--- Масштабирование признаков (MinMaxScaler) ---")
scaler = MinMaxScaler(feature_range=(0, 1)) # Используем стандартный диапазон
# Обучаем scaler ТОЛЬКО на обучающих признаках X_train
scaler.fit(X_train)

# Применяем scaler к обучающей и тестовой выборкам для получения X_train_m, X_test_m
X_train_m = scaler.transform(X_train)
X_test_m = scaler.transform(X_test)

# Преобразуем обратно в DataFrame для удобства (опционально, но полезно)
X_train_m = pd.DataFrame(X_train_m, index=X_train.index, columns=X_train.columns)
X_test_m = pd.DataFrame(X_test_m, index=X_test.index, columns=X_test.columns)

print("Масштабирование выполнено.")
print("Созданы X_train_m и X_test_m.")
# print("Пример X_train_m:", X_train_m.head())

In [ ]:
# 4. Настройка TimeSeriesSplit для кросс-валидации (код тот же)
print("\n--- Настройка TimeSeriesSplit ---")

tscv = TimeSeriesSplit(n_splits=n_cv_splits)
print(f"TimeSeriesSplit настроен с {tscv.get_n_splits()} сплитами.")

In [ ]:
import pandas as pd
import numpy as np

# --- Настройки (копируем из блока Feature Engineering) ---
lags_to_create = [1,2, 3, 6, 12]
rolling_windows = [3, 6, 12]
# Имена колонок признаков (в том порядке, как они в X_train)
# Важно: Получи этот порядок из твоего готового X_train
# Например: feature_order = X_train.columns.tolist()
# Пока зададим вручную на основе нашего кода:
feature_order = [
    't-1','t-2', 't-3', 't-6', 't-12',
    't_mean_lag3', 't_std_lag3',
    't_mean_lag6', 't_std_lag6',
    't_mean_lag12', 't_std_lag12',
    'month_sin', 'month_cos'
]
# --- Конец Настроек ---


def calculate_features_for_step(history_series: pd.Series):
    """
    Рассчитывает набор признаков для прогноза СЛЕДУЮЩЕГО шага
    на основе предоставленной истории.

    Args:
        history_series (pd.Series): Временной ряд значений (y) с DatetimeIndex,
                                     включающий все данные до момента,
                                     *предшествующего* тому, для которого нужны признаки.
                                     Последняя точка - это y(t-1).

    Returns:
        pd.Series: Строка признаков для прогнозирования y(t),
                   индексированная именами признаков в правильном порядке.
                   Или None, если истории недостаточно.
    """
    features = {}
    n = len(history_series)
    next_index = history_series.index[-1] + pd.DateOffset(months=1)

    # Рассчитываем лаги (относительно ПОСЛЕДНЕЙ точки в history_series, т.е. y(t-1))
    for lag in lags_to_create:
        if n >= lag:
            features[f't-{lag}'] = history_series.iloc[-lag]
        else:
            features[f't-{lag}'] = np.nan # Недостаточно истории

    # Рассчитываем скользящие статистики (окно ЗАКАНЧИВАЕТСЯ на последней точке history_series)
    # Это соответствует .shift(1) при создании обучающей выборки
    for window in rolling_windows:
        if n >= window:
            window_data = history_series.iloc[-window:]
            features[f't_mean_lag{window}'] = window_data.mean()
            features[f't_std_lag{window}'] = window_data.std()
        else:
            features[f't_mean_lag{window}'] = np.nan
            features[f't_std_lag{window}'] = np.nan

    # Рассчитываем циклические признаки для СЛЕДУЮЩЕГО месяца
    next_month_num = next_index.month
    features['month_sin'] = np.sin(2 * np.pi * next_month_num / 12)
    features['month_cos'] = np.cos(2 * np.pi * next_month_num / 12)

    # Собираем результат в Series и проверяем на NaN
    features_series = pd.Series(features)
    if features_series.isnull().any():
        # print(f"Warning: NaN в признаках для индекса {next_index}. История слишком коротка?")
        # Решаем, как обрабатывать NaN - пока вернем None
        # В рекурсивном прогнозе NaN быть не должно, т.к. стартуем с полной истории
        return None

    # Возвращаем признаки в правильном порядке
    try:
        return features_series[feature_order]
    except KeyError as e:
        print(f"Ошибка: Несовпадение имен признаков! {e}")
        print("Ожидаемый порядок:", feature_order)
        print("Сгенерированные признаки:", features_series.index.tolist())
        return None


# --- Тестирование функции ---
print("--- Тестирование функции calculate_features_for_step ---")
# Возьмем конец y_train для примера
test_history = y_train.iloc[-15:] # Последние 15 точек обучающей выборки
print("Последние точки истории для теста функции:")
print(test_history)

# Рассчитаем признаки для ПЕРВОГО шага прогноза (т.е. для первого индекса в y_test)
first_step_features = calculate_features_for_step(test_history)

print("\nПризнаки, рассчитанные для первого шага прогноза:")
if first_step_features is not None:
    print(first_step_features.round(4))
    # Сравним с первой строкой X_test (должны быть очень близки, если test_history - это конец y_train)
    print("\nПервая строка X_test (для сравнения):")
    print(X_test.iloc[0].round(4))

    # Проверим совпадение (допускаем небольшие погрешности вычислений)
    if np.allclose(first_step_features.fillna(0), X_test.iloc[0].fillna(0), atol=1e-6):
         print("\nТест пройден: Рассчитанные признаки совпадают с первой строкой X_test.")
    else:
         print("\nТест НЕ пройден: Рассчитанные признаки НЕ совпадают с первой строкой X_test.")
         # Выведем разницу для отладки
         # print("Разница:")
         # print((first_step_features - X_test.iloc[0]).round(4))

else:
    print("Функция вернула None, возможно, истории недостаточно.")

In [ ]:
def recursive_predict(model, initial_history_series: pd.Series, n_steps: int,
                      feature_calculator, feature_order: list, scaler=None):
    """
    Генерирует рекурсивный многошаговый прогноз.

    Args:
        model: Обученная ML модель (с методом predict).
        initial_history_series (pd.Series): Исходная история y до начала прогноза.
        n_steps (int): Количество шагов прогноза.
        feature_calculator (function): Функция, рассчитывающая признаки для шага.
        feature_order (list): Порядок колонок признаков, ожидаемый моделью.
        scaler (MinMaxScaler, optional): Scaler для масштабирования признаков
                                          и обратного масштабирования прогноза,
                                          если модель обучалась на scaled данных.
                                          Defaults to None.

    Returns:
        np.array: Массив с прогнозами на n_steps шагов.
    """
    # Копируем историю, чтобы не изменять оригинал
    current_history = initial_history_series.copy()
    predictions = []

    print(f"Запуск рекурсивного прогноза для {model.__class__.__name__} на {n_steps} шагов...")

    for i in range(n_steps):
        # 1. Рассчитать признаки для текущего шага (прогноза y(t))
        #    на основе истории до t-1
        features_for_step = feature_calculator(current_history)

        if features_for_step is None:
            print(f"Ошибка: Не удалось рассчитать признаки на шаге {i+1}. Прерывание.")
            # Заполняем оставшиеся прогнозы NaN или последним значением
            predictions.extend([predictions[-1] if predictions else np.nan] * (n_steps - i))
            break

        # 2. Подготовить признаки для модели
        # Преобразуем в DataFrame с одной строкой и нужным порядком колонок
        features_df = features_for_step.to_frame().T
        # Масштабируем, если scaler предоставлен
        if scaler:
            features_scaled = scaler.transform(features_df)
            model_input = features_scaled
        else:
            model_input = features_df # Используем исходные признаки

        # 3. Сделать прогноз на 1 шаг вперед
        try:
             # model.predict ожидает 2D массив
            next_pred = model.predict(model_input)[0]
        except Exception as e:
             print(f"Ошибка предсказания модели на шаге {i+1}: {e}")
             # Заполняем NaN или последним значением
             next_pred_final = predictions[-1] if predictions else np.nan
             predictions.extend([next_pred_final] * (n_steps - i))
             break

        # 4. Обратно масштабировать прогноз НЕ НУЖНО, если модель предсказывает y
        #    Масштабирование/обратное масштабирование применяется к ПРИЗНАКАМ (X),
        #    а не к целевой переменной (y) внутри этого цикла.
        #    ОБУЧЕНИЕ модели происходило на y_train (немасштабированном)
        #    или на y_train_lstm (масштабированном для LSTM - это особый случай).
        #    !!! ВАЖНО: Убедимся, что все наши ML модели обучались на y_train !!!
        #    Да, Ridge, Lasso, RF, XGBoost обучались на y_train. LSTM - особый случай.
        next_pred_final = next_pred # Прогноз уже в нужном масштабе

        # 5. Сохранить прогноз
        predictions.append(next_pred_final)

        # 6. Добавить прогноз к истории для следующего шага
        next_index = current_history.index[-1] + pd.DateOffset(months=1)
        # Используем pd.concat
        current_history = pd.concat([current_history, pd.Series([next_pred_final], index=[next_index])])

        # Вывод прогресса (опционально)
        # if (i + 1) % 6 == 0:
        #      print(f"  Шаг {i+1}/{n_steps} выполнен.")

    print(f"Рекурсивный прогноз для {model.__class__.__name__} завершен.")
    return np.array(predictions)


# Ridge

In [ ]:
# --- БЛОК 7: Модель Ridge (Обучение с hyperopt + Рекурсивный Прогноз) ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
# Убираем GridSearchCV, добавляем hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error
# Убедимся, что tscv определен из предыдущих блоков
# from sklearn.model_selection import TimeSeriesSplit

# --- ПРЕДПОЛАГАЕМ, ЧТО ПЕРЕМЕННЫЕ СУЩЕСТВУЮТ ИЗ ПРЕДЫДУЩИХ БЛОКОВ ---
# X_train_m, y_train - масштабированные DataFrame и Series для обучения
# X_test_m - масштабированный DataFrame для теста (если scaler используется в recursive_predict)
# y_test - Series с реальными значениями на тесте
# tscv - настроенный TimeSeriesSplit
# scaler - обученный MinMaxScaler (если используется)
# calculate_features_for_step - функция расчета признаков
# feature_order - список с порядком признаков
# test_months - количество месяцев в тесте (24)
# horizons_to_evaluate - список горизонтов [1, 2, 3, 6, 12, 18, 24]
# result_rmse, df_horizon_rmse: Инициализированные DataFrames для результатов
# recursive_forecasts: Инициализированный словарь для прогнозов
# plot_start_date: Дата начала для графиков прогнозов
# forecast_arima: Прогноз ARIMA (для графика)
# train_values, test_values - исходные (немасштабированные) ряды для графиков
# random_seed - для воспроизводимости (например, 42)
# recursive_predict - функция рекурсивного прогноза
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# --- Дополнительные настройки ---
model_name_ridge = "Ridge_Recursive" # Новое имя
MAX_EVALS_HYPEROPT = 50 # Количество итераций для hyperopt (можно изменить)
# --- Конец Дополнительных настроек ---

print(f"\n--- Обучение и Прогнозирование: {model_name_ridge} ---")

# 1. Определяем пространство поиска для hyperopt
#    Используем loguniform, так как alpha может меняться на порядки и > 0.
#    Задаем широкий диапазон, например, от 1e-4 до 100 (в логарифмах)
space_ridge = {
    'alpha': hp.loguniform('alpha', np.log(0.0001), np.log(100.0))
}

# 2. Определяем целевую функцию (objective) для hyperopt
def objective_ridge(params):
    """
    Целевая функция для hyperopt. Обучает Ridge с заданным alpha
    и оценивает его с помощью TimeSeriesSplit кросс-валидации.
    Возвращает средний RMSE по фолдам.
    """
    current_alpha = params['alpha']

    model = Ridge(alpha=current_alpha)

    rmses = []
    # Цикл кросс-валидации по фолдам TimeSeriesSplit
    # Убедись, что X_train_m и y_train - это те данные, на которых ты хочешь валидироваться
    # и что они имеют совместимые типы (например, оба Pandas или оба NumPy)
    try:
        for train_idx, val_idx in tscv.split(X_train_m):
            # Используем .iloc, если X_train_m и y_train - Pandas объекты
            X_fold_train, X_fold_val = X_train_m.iloc[train_idx], X_train_m.iloc[val_idx]
            y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

            model.fit(X_fold_train, y_fold_train)
            preds = model.predict(X_fold_val)

            # Обработка возможных NaN или бесконечностей в прогнозах или реальных значениях
            if not np.all(np.isfinite(preds)) or not np.all(np.isfinite(y_fold_val.values)):
                 print(f"Warning: Non-finite values found in fold predictions or actuals for alpha={current_alpha}. Skipping fold.")
                 # Можно вернуть очень большое значение ошибки, чтобы hyperopt избегал таких параметров
                 # return {'loss': 1e10, 'status': STATUS_OK, 'params': params}
                 # Или просто пропустить фолд, но это исказит среднее
                 continue # Пропускаем этот фолд

            # Убедимся, что y_fold_val не пустой
            if len(y_fold_val) == 0:
                print(f"Warning: Empty validation fold encountered for alpha={current_alpha}. Skipping fold.")
                continue

            rmse = np.sqrt(mean_squared_error(y_fold_val, preds))
            rmses.append(rmse)

        if not rmses: # Если все фолды были пропущены
            print(f"Warning: No valid RMSE scores calculated for alpha={current_alpha}. Returning large error.")
            avg_rmse = 1e10 # Возвращаем большое значение
        else:
            avg_rmse = np.mean(rmses)

    except Exception as e:
        print(f"Error during cross-validation for alpha={current_alpha}: {e}")
        avg_rmse = 1e10 # Возвращаем большое значение при ошибке

    # Проверка на NaN или inf в итоговом avg_rmse
    if not np.isfinite(avg_rmse):
        avg_rmse = 1e10

    # Возвращаем словарь, ожидаемый hyperopt
    return {'loss': avg_rmse, 'status': STATUS_OK, 'params': params}

# 3. Запускаем оптимизацию hyperopt
print(f"Запуск hyperopt для подбора alpha (max_evals={MAX_EVALS_HYPEROPT})...")
trials_ridge = Trials() # Объект для хранения истории поиска

# Установка random seed для воспроизводимости hyperopt
# Используй один из вариантов в зависимости от версии Python/numpy
try:
    rstate = np.random.default_rng(random_seed)
except AttributeError:
    rstate = np.random.RandomState(random_seed)


best_params_ridge = fmin(
    fn=objective_ridge,       # Наша целевая функция
    space=space_ridge,        # Пространство поиска параметров
    algo=tpe.suggest,         # Алгоритм оптимизации TPE
    max_evals=MAX_EVALS_HYPEROPT, # Количество итераций
    trials=trials_ridge,      # Для логирования
    rstate=rstate             # Генератор случайных чисел для воспроизводимости
)

# Извлекаем лучший результат из trials (лучший loss = RMSE на CV)
best_cv_rmse = trials_ridge.best_trial['result']['loss']
# fmin возвращает только значения параметров, не их имена, восстанавливаем полный словарь
best_final_params_ridge = {'alpha': best_params_ridge['alpha']}

print(f"\nЛучшее найденное значение alpha для Ridge: {best_final_params_ridge['alpha']:.6f}") # Выводим точнее
print(f"Лучший RMSE на кросс-валидации: {best_cv_rmse:.4f}")

# 4. Обучаем финальную модель Ridge с лучшими найденными параметрами
print("\nОбучение финальной модели Ridge на всем X_train_m...")
final_ridge_model = Ridge(**best_final_params_ridge) # Используем двойную звездочку для распаковки словаря
final_ridge_model.fit(X_train_m, y_train)
print("Финальная модель обучена.")

# 5. Генерация рекурсивного прогноза (используя final_ridge_model)
print("\nГенерация рекурсивного прогноза...")
initial_history_ridge = y_train.copy()
forecast_ridge_recursive = recursive_predict(
    model=final_ridge_model, # <<<--- Используем финальную модель
    initial_history_series=initial_history_ridge,
    n_steps=test_months,
    feature_calculator=calculate_features_for_step,
    feature_order=feature_order,
    scaler=scaler # Передаем scaler, если он нужен для прогноза
)
# Обновляем словарь прогнозов с новым именем модели
if model_name_ridge in recursive_forecasts: del recursive_forecasts[model_name_ridge] # Удаляем старый прогноз Ridge
recursive_forecasts[model_name_ridge] = forecast_ridge_recursive # Добавляем новый
print(f"Прогноз {model_name_ridge} сгенерирован и добавлен в словарь.")

# 6. Оценка RMSE (код остается почти тем же, но использует новые прогнозы)
# ... (код проверки длин y_test_eval, forecast_ridge_eval) ...
if len(forecast_ridge_recursive) != len(y_test):
     min_len = min(len(forecast_ridge_recursive), len(y_test))
     print(f"Warning: Length mismatch in recursive forecast ({len(forecast_ridge_recursive)}) vs y_test ({len(y_test)}). Using length {min_len}.")
     y_test_eval = y_test[:min_len]; forecast_ridge_eval = forecast_ridge_recursive[:min_len]
else:
     y_test_eval = y_test; forecast_ridge_eval = forecast_ridge_recursive

overall_rmse_ridge_recursive = np.sqrt(mean_squared_error(y_test_eval, forecast_ridge_eval))
print(f"\nОбщий RMSE на тесте для {model_name_ridge}: {overall_rmse_ridge_recursive:.4f}")

# 7. Обновление таблиц результатов (код остается тем же, но с новым model_name_ridge)
# Удаляем старые результаты Ridge перед добавлением новых
result_rmse = result_rmse[~result_rmse['method'].str.contains('Ridge', case=False)]
df_horizon_rmse = df_horizon_rmse[~df_horizon_rmse['method'].str.contains('Ridge', case=False)]

result_ridge_entry = {'method': model_name_ridge, 'rmse': overall_rmse_ridge_recursive}
result_rmse = pd.concat([result_rmse, pd.DataFrame([result_ridge_entry])], ignore_index=True)

print("RMSE по горизонтам:")
horizon_results_list_ridge_rec = []
for h in horizons_to_evaluate:
    if h <= len(forecast_ridge_eval): # Используем длину фактического прогноза
        forecast_h_step = forecast_ridge_eval[:h]
        actual_h_step = y_test_eval[:h]
        rmse_h = np.sqrt(mean_squared_error(actual_h_step, forecast_h_step))
        print(f"  1-{h} мес.: {rmse_h:.4f}")
        horizon_results_list_ridge_rec.append({'method': model_name_ridge, 'horizon': h, 'rmse': rmse_h})
df_horizon_rmse_ridge_rec = pd.DataFrame(horizon_results_list_ridge_rec)
df_horizon_rmse = pd.concat([df_horizon_rmse, df_horizon_rmse_ridge_rec], ignore_index=True)
print("Результаты RMSE по горизонтам добавлены/обновлены.")

# 8. График прогноза (код остается тем же, но с новым model_name_ridge и прогнозами)
# ... (код графика) ...
plt.figure(figsize=(12, 6))
history_to_plot = train_values[train_values.index >= plot_start_date]
plt.plot(history_to_plot.index, history_to_plot, label='Обучающая выборка (часть)')
plt.plot(y_test_eval.index, y_test_eval, label='Тестовая выборка (реальные)', color='orange', linewidth=2) # Используем y_test_eval
forecast_ridge_series = pd.Series(forecast_ridge_eval, index=y_test_eval.index[:len(forecast_ridge_eval)])
plt.plot(forecast_ridge_series.index, forecast_ridge_series, label=f'Прогноз {model_name_ridge}', color='green', linestyle='--')
# Добавляем ARIMA для сравнения
forecast_arima_series_plot = pd.Series(forecast_arima, index=y_test_eval.index[:len(forecast_arima)])
plt.plot(forecast_arima_series_plot.index, forecast_arima_series_plot, label='Прогноз ARIMA', color='blue', linestyle=':', alpha=0.7)
plt.title(f'Прогноз ИПЦ с помощью {model_name_ridge}')
plt.xlabel('Дата'); plt.ylabel('ИПЦ (месячный рост)'); plt.legend(); plt.grid(True)
plt.show()


# 9. График RMSE vs Горизонт (код остается тем же, но с новым model_name_ridge)
# ... (код графика) ...
if not df_horizon_rmse_ridge_rec.empty:
    plt.figure(figsize=(10, 5))
    plt.plot(df_horizon_rmse_ridge_rec['horizon'], df_horizon_rmse_ridge_rec['rmse'], marker='o', linestyle='-')
    plt.title(f'Зависимость RMSE от горизонта прогноза для {model_name_ridge}')
    plt.xlabel('Горизонт прогноза (месяцы)'); plt.ylabel('RMSE')
    plt.xticks(horizons_to_evaluate); plt.grid(True); plt.show()
else:
     print(f"Нет данных для построения графика RMSE vs Горизонт для {model_name_ridge}")


print(f"--- Анализ {model_name_ridge} завершен ---")

# 10. Вывод итоговых таблиц (код остается тем же)
# ... (код вывода таблиц) ...
print("\nТекущая таблица общих RMSE:")
print(result_rmse.sort_values(by='rmse'))
print("\nТекущая таблица RMSE по горизонтам:")
pivot_table_current_ridge_ho = df_horizon_rmse.pivot(index='horizon', columns='method', values='rmse')
current_order_ridge_ho = result_rmse.sort_values(by='rmse')['method'].tolist()
for method in current_order_ridge_ho:
     if method not in pivot_table_current_ridge_ho.columns: pivot_table_current_ridge_ho[method] = np.nan
if model_name_ridge not in current_order_ridge_ho: current_order_ridge_ho.append(model_name_ridge)
try:
    print(pivot_table_current_ridge_ho[current_order_ridge_ho].round(4))
except KeyError as e:
     print(f"Ошибка при сортировке колонок для вывода RMSE по горизонтам: {e}. Вывод как есть:")
     print(pivot_table_current_ridge_ho.round(4))

# Lasso

In [ ]:
# --- БЛОК 8: Модель Lasso (Обучение с hyperopt + Рекурсивный Прогноз) ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
# Убираем GridSearchCV, добавляем hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error
# Убедимся, что tscv определен
# from sklearn.model_selection import TimeSeriesSplit

# --- ПРЕДПОЛАГАЕМ, ЧТО ПЕРЕМЕННЫЕ СУЩЕСТВУЮТ ИЗ ПРЕДЫДУЩИХ БЛОКОВ ---
# X_train_m, y_train - масштабированные DataFrame и Series
# X_test_m, y_test - для теста
# tscv - настроенный TimeSeriesSplit
# scaler - обученный MinMaxScaler
# calculate_features_for_step, feature_order, test_months, horizons_to_evaluate
# result_rmse, df_horizon_rmse, recursive_forecasts
# plot_start_date, forecast_arima
# train_values, test_values
# random_seed
# recursive_predict
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# --- Дополнительные настройки ---
model_name_lasso = "Lasso_Recursive" # <<<--- СОХРАНЯЕМ ИМЯ
MAX_EVALS_HYPEROPT_LASSO = 50 # Количество итераций для hyperopt Lasso (можно изменить)
LASSO_MAX_ITER = 10000 # Увеличиваем итерации для сходимости Lasso
# --- Конец Дополнительных настроек ---

print(f"\n--- Обучение и Прогнозирование: {model_name_lasso} ---")

# 1. Определяем пространство поиска для hyperopt Lasso
#    Используем loguniform от 1e-5 до 1.0.
space_lasso = {
    # np.log(1e-5) ~ -11.5, np.log(1.0) = 0
    'alpha': hp.loguniform('alpha', np.log(0.00001), np.log(1.0))
}
print(f"Пространство поиска alpha для Lasso: от {np.exp(np.log(0.00001)):.1E} до {np.exp(np.log(1.0)):.1f} (loguniform)")

# 2. Определяем целевую функцию (objective) для hyperopt Lasso
def objective_lasso(params):
    """
    Целевая функция для hyperopt. Обучает Lasso с заданным alpha
    и оценивает его с помощью TimeSeriesSplit кросс-валидации.
    Возвращает средний RMSE по фолдам.
    """
    current_alpha = params['alpha']

    # Увеличиваем max_iter для Lasso
    # Можно добавить `tol` для контроля точности, если модель долго сходится
    model = Lasso(alpha=current_alpha, max_iter=LASSO_MAX_ITER, tol=0.001, random_state=random_seed)

    rmses = []
    try:
        for train_idx, val_idx in tscv.split(X_train_m): # X_train_m - масштабированные признаки
            X_fold_train, X_fold_val = X_train_m.iloc[train_idx], X_train_m.iloc[val_idx]
            y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

            model.fit(X_fold_train, y_fold_train)
            preds = model.predict(X_fold_val)

            if not np.all(np.isfinite(preds)) or not np.all(np.isfinite(y_fold_val.values)):
                 # print(f"Warning: Non-finite values found for alpha={current_alpha:.6f}. Skipping fold.") # Можно раскомментировать для отладки
                 continue

            if len(y_fold_val) == 0:
                 # print(f"Warning: Empty validation fold for alpha={current_alpha:.6f}. Skipping fold.")
                 continue

            rmse = np.sqrt(mean_squared_error(y_fold_val, preds))
            rmses.append(rmse)

        if not rmses:
            # print(f"Warning: No valid RMSE scores for alpha={current_alpha:.6f}. Returning large error.")
            avg_rmse = 1e10
        else:
            avg_rmse = np.mean(rmses)

    except Exception as e:
        # print(f"Error during CV for alpha={current_alpha:.6f}: {e}") # Можно раскомментировать для отладки
        avg_rmse = 1e10 # Возвращаем большое значение при ошибке

    if not np.isfinite(avg_rmse):
        avg_rmse = 1e10

    return {'loss': avg_rmse, 'status': STATUS_OK, 'params': params}

# 3. Запускаем оптимизацию hyperopt
print(f"Запуск hyperopt для подбора alpha Lasso (max_evals={MAX_EVALS_HYPEROPT_LASSO})...")
trials_lasso = Trials()

# Установка random seed для воспроизводимости hyperopt
try:
    rstate_lasso = np.random.default_rng(random_seed)
except AttributeError:
    rstate_lasso = np.random.RandomState(random_seed)

best_params_lasso = fmin(
    fn=objective_lasso,
    space=space_lasso,
    algo=tpe.suggest,
    max_evals=MAX_EVALS_HYPEROPT_LASSO,
    trials=trials_lasso,
    rstate=rstate_lasso,
    show_progressbar=True # Показываем прогресс
)

# Извлекаем лучший результат из trials
try:
    best_cv_rmse_lasso = trials_lasso.best_trial['result']['loss']
    # fmin возвращает только значения параметров, не их имена
    best_final_params_lasso = {'alpha': best_params_lasso['alpha']}
    print(f"\nЛучшее найденное значение alpha для Lasso: {best_final_params_lasso['alpha']:.6f}")
    print(f"Лучший RMSE на кросс-валидации: {best_cv_rmse_lasso:.4f}")
except Exception as e:
    print(f"\nНе удалось извлечь лучший результат из hyperopt trials: {e}")
    print("Возможно, все итерации завершились с ошибкой. Проверьте objective функцию.")
    # В этом случае прерываем, так как нет лучших параметров
    raise SystemExit("Прерывание из-за ошибки в hyperopt.")


# 4. Обучаем финальную модель Lasso с лучшими найденными параметрами
print("\nОбучение финальной модели Lasso на всем X_train_m...")
final_lasso_model = Lasso(**best_final_params_lasso, max_iter=LASSO_MAX_ITER, tol=0.001, random_state=random_seed)
final_lasso_model.fit(X_train_m, y_train)

# Выводим кол-во отобранных признаков финальной моделью
n_features_selected = np.sum(final_lasso_model.coef_ != 0)
print(f"Количество отобранных признаков финальной моделью: {n_features_selected} из {X_train_m.shape[1]}")
print("Финальная модель обучена.")

# 5. Генерация рекурсивного прогноза (используя final_lasso_model)
print("\nГенерация рекурсивного прогноза...")
initial_history_lasso = y_train.copy()
forecast_lasso_recursive = recursive_predict(
    model=final_lasso_model, # <<<--- Используем финальную модель
    initial_history_series=initial_history_lasso,
    n_steps=test_months,
    feature_calculator=calculate_features_for_step,
    feature_order=feature_order,
    scaler=scaler # Передаем scaler
)
# Обновляем словарь прогнозов, сохраняя имя модели
if model_name_lasso in recursive_forecasts: del recursive_forecasts[model_name_lasso]
recursive_forecasts[model_name_lasso] = forecast_lasso_recursive
print(f"Прогноз {model_name_lasso} сгенерирован и добавлен в словарь.")

# 6. Оценка RMSE (код остается тем же)
# ... (код проверки длин y_test_eval, forecast_lasso_eval) ...
if len(forecast_lasso_recursive) != len(y_test):
     min_len = min(len(forecast_lasso_recursive), len(y_test))
     print(f"Warning: Length mismatch in recursive forecast ({len(forecast_lasso_recursive)}) vs y_test ({len(y_test)}). Using length {min_len}.")
     y_test_eval = y_test[:min_len]; forecast_lasso_eval = forecast_lasso_recursive[:min_len]
else:
     y_test_eval = y_test; forecast_lasso_eval = forecast_lasso_recursive

overall_rmse_lasso_recursive = np.sqrt(mean_squared_error(y_test_eval, forecast_lasso_eval))
print(f"\nОбщий RMSE на тесте для {model_name_lasso}: {overall_rmse_lasso_recursive:.4f}")

# 7. Обновление таблиц результатов (код остается тем же)
# Удаляем старые результаты Lasso перед добавлением новых
result_rmse = result_rmse[~result_rmse['method'].str.contains('Lasso', case=False)]
df_horizon_rmse = df_horizon_rmse[~df_horizon_rmse['method'].str.contains('Lasso', case=False)]

result_lasso_entry = {'method': model_name_lasso, 'rmse': overall_rmse_lasso_recursive}
result_rmse = pd.concat([result_rmse, pd.DataFrame([result_lasso_entry])], ignore_index=True)

print("RMSE по горизонтам:")
horizon_results_list_lasso_rec = []
for h in horizons_to_evaluate:
    if h <= len(forecast_lasso_eval): # Используем длину фактического прогноза
        forecast_h_step = forecast_lasso_eval[:h]
        actual_h_step = y_test_eval[:h]
        rmse_h = np.sqrt(mean_squared_error(actual_h_step, forecast_h_step))
        print(f"  1-{h} мес.: {rmse_h:.4f}")
        horizon_results_list_lasso_rec.append({'method': model_name_lasso, 'horizon': h, 'rmse': rmse_h})
df_horizon_rmse_lasso_rec = pd.DataFrame(horizon_results_list_lasso_rec)
df_horizon_rmse = pd.concat([df_horizon_rmse, df_horizon_rmse_lasso_rec], ignore_index=True)
print("Результаты RMSE по горизонтам добавлены/обновлены.")

# 4. График прогноза
plt.figure(figsize=(12, 6))
history_to_plot = train_values[train_values.index >= plot_start_date]
plt.plot(history_to_plot.index, history_to_plot, label='Обучающая выборка (часть)')
plt.plot(test.index, test_values, label='Тестовая выборка (реальные)', color='orange', linewidth=2)
forecast_lasso_series = pd.Series(forecast_lasso_eval, index=test.index[:len(forecast_lasso_eval)])
plt.plot(forecast_lasso_series.index, forecast_lasso_series, label=f'Прогноз {model_name_lasso}', color='purple', linestyle='--')
forecast_arima_series = pd.Series(forecast_arima, index=test.index[:len(forecast_arima)])
plt.plot(forecast_arima_series.index, forecast_arima_series, label='Прогноз ARIMA', color='blue', linestyle=':', alpha=0.7)
plt.title(f'Прогноз ИПЦ с помощью {model_name_lasso} (Рекурсивный)')
plt.xlabel('Дата'); plt.ylabel('ИПЦ (месячный рост)'); plt.legend(); plt.grid(True)
plt.show()

# 5. График RMSE vs Горизонт (только для этой модели)
plt.figure(figsize=(10, 5))
plt.plot(df_horizon_rmse_lasso_rec['horizon'], df_horizon_rmse_lasso_rec['rmse'], marker='o', linestyle='-')
plt.title(f'Зависимость RMSE от горизонта прогноза для {model_name_lasso}')
plt.xlabel('Горизонт прогноза (месяцы)'); plt.ylabel('RMSE')
plt.xticks(horizons_to_evaluate); plt.grid(True); plt.show()

print(f"--- Анализ {model_name_lasso} завершен ---")

# 6. Вывод итоговых таблиц (для сравнения с предыдущими)
print("\nТекущая таблица общих RMSE:")
print(result_rmse.sort_values(by='rmse'))
print("\nТекущая таблица RMSE по горизонтам:")
pivot_table_current = df_horizon_rmse.pivot(index='horizon', columns='method', values='rmse')
current_order = result_rmse['method'].tolist()
for method in current_order:
     if method not in pivot_table_current.columns: pivot_table_current[method] = np.nan
print(pivot_table_current[current_order].round(4))

In [ ]:
# 4. График прогноза
plt.figure(figsize=(12, 6))
history_to_plot = train_values[train_values.index >= plot_start_date]
plt.plot(history_to_plot.index, history_to_plot, label='Обучающая выборка (часть)')
plt.plot(test.index, test_values, label='Тестовая выборка (реальные)', color='orange', linewidth=2)
forecast_lasso_series = pd.Series(forecast_lasso_eval, index=test.index[:len(forecast_lasso_eval)])
plt.plot(forecast_lasso_series.index, forecast_lasso_series, label=f'Прогноз {model_name_lasso}', color='purple', linestyle='--')
forecast_ridge_series = pd.Series(forecast_ridge_eval, index=y_test_eval.index[:len(forecast_ridge_eval)])
plt.plot(forecast_ridge_series.index, forecast_ridge_series, label=f'Прогноз {model_name_ridge}', color='green', linestyle='--')
forecast_arima_series = pd.Series(forecast_arima, index=test.index[:len(forecast_arima)])
plt.plot(forecast_arima_series.index, forecast_arima_series, label='Прогноз ARIMA', color='blue', linestyle=':', alpha=0.7)
plt.title(f'Прогноз ИПЦ с помощью Ridge и Lasso (Рекурсивный)')
plt.xlabel('Дата'); plt.ylabel('ИПЦ (месячный рост)'); plt.legend(); plt.grid(True)
plt.show()

#RF

In [ ]:
# --- БЛОК 9: Модель Random Forest (Обучение с hyperopt + Рекурсивный Прогноз) ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
# Убираем GridSearchCV, добавляем hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from sklearn.metrics import mean_squared_error
# Убедимся, что tscv определен
# from sklearn.model_selection import TimeSeriesSplit

# --- ПРЕДПОЛАГАЕМ, ЧТО ПЕРЕМЕННЫЕ СУЩЕСТВУЮТ ИЗ ПРЕДЫДУЩИХ БЛОКОВ ---
# X_train, y_train - НЕмасштабированные DataFrame и Series
# X_test, y_test - для теста
# tscv - настроенный TimeSeriesSplit
# calculate_features_for_step, feature_order, test_months, horizons_to_evaluate
# result_rmse, df_horizon_rmse, recursive_forecasts
# plot_start_date, forecast_arima
# train_values, test_values
# random_seed
# recursive_predict
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# --- Дополнительные настройки ---
model_name_rf = "RF_Recursive" # <<<--- СОХРАНЯЕМ ИМЯ
MAX_EVALS_HYPEROPT_RF = 50 # Количество итераций для hyperopt RF (можно увеличить до 75-100, если время позволяет)
# --- Конец Дополнительных настроек ---

print(f"\n--- Обучение и Прогнозирование: {model_name_rf} ---")

# 1. Определяем пространство поиска для hyperopt Random Forest
#    Определяем диапазоны и типы распределений для каждого параметра
space_rf = {
    # hp.quniform(label, low, high, q) - выбирает значение round(uniform(low, high) / q) * q
    'n_estimators': hp.quniform('n_estimators', 100, 300, 25), # Шаг 25, от 100 до 300
    # hp.choice(label, options) - выбирает один из списка options
    'max_depth': hp.choice('max_depth', [4, 6, 8, 10, 12, None]), # Включаем None и значения вокруг 6
    'max_features': hp.choice('max_features', ['sqrt', 0.6, 0.7, 0.8, 0.9]), # Включаем 'sqrt' и доли вокруг 0.7
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1), # Целые от 2 до 10
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 12, 2),    # Целые от 1 до 6
    # 'bootstrap': hp.choice('bootstrap', [True, False]) # Можно добавить, если нужно
}
print(f"Пространство поиска для Random Forest: {space_rf}")

# 2. Определяем целевую функцию (objective) для hyperopt RF
def objective_rf(params):
    """
    Целевая функция для hyperopt. Обучает RandomForestRegressor с заданными
    параметрами и оценивает его с помощью TimeSeriesSplit кросс-валидации.
    Возвращает средний RMSE по фолдам.
    """
    # Преобразуем параметры из hyperopt в нужный формат (особенно целые числа)
    params['n_estimators'] = int(params['n_estimators'])
    # max_depth может быть None, hp.choice вернет его как есть
    # max_features может быть строкой или float, hp.choice вернет как есть
    params['min_samples_split'] = int(params['min_samples_split'])
    params['min_samples_leaf'] = int(params['min_samples_leaf'])

    # Создаем модель с текущими параметрами + важные фиксированные параметры
    model = RandomForestRegressor(
        **params,                 # Распаковываем параметры из hyperopt
        random_state=random_seed,
        n_jobs=-1                 # Используем все ядра CPU
    )

    rmses = []
    try:
        # Используем НЕмасштабированные X_train
        for train_idx, val_idx in tscv.split(X_train):
            X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

            model.fit(X_fold_train, y_fold_train)
            preds = model.predict(X_fold_val)

            if not np.all(np.isfinite(preds)) or not np.all(np.isfinite(y_fold_val.values)):
                 continue

            if len(y_fold_val) == 0:
                 continue

            rmse = np.sqrt(mean_squared_error(y_fold_val, preds))
            rmses.append(rmse)

        if not rmses:
            avg_rmse = 1e10
        else:
            avg_rmse = np.mean(rmses)

    except Exception as e:
        # print(f"Error during CV for params={params}: {e}") # Отладка
        avg_rmse = 1e10

    if not np.isfinite(avg_rmse):
        avg_rmse = 1e10

    # Сохраняем параметры в trials для последующего анализа (не обязательно, но полезно)
    # hyperopt автоматически сохраняет loss и status
    # return {'loss': avg_rmse, 'status': STATUS_OK, 'params': params}
    # Но можно вернуть только loss и status, т.к. fmin вернет лучшие параметры
    return {'loss': avg_rmse, 'status': STATUS_OK }


# 3. Запускаем оптимизацию hyperopt
print(f"Запуск hyperopt для подбора параметров Random Forest (max_evals={MAX_EVALS_HYPEROPT_RF})...")
trials_rf = Trials()

try:
    rstate_rf = np.random.default_rng(random_seed)
except AttributeError:
    rstate_rf = np.random.RandomState(random_seed)

# fmin вернет словарь только с индексами/значениями из hp.choice или числовыми значениями
best_params_raw = fmin(
    fn=objective_rf,
    space=space_rf,
    algo=tpe.suggest,
    max_evals=MAX_EVALS_HYPEROPT_RF,
    trials=trials_rf,
    rstate=rstate_rf,
    show_progressbar=True
)

# Преобразуем результат fmin обратно в читаемые параметры, используя space
# space_eval нужен для hp.choice, чтобы получить реальное значение (строку или None), а не индекс
best_final_params_rf = space_eval(space_rf, best_params_raw)

# Преобразуем числовые параметры к нужному типу (int)
best_final_params_rf['n_estimators'] = int(best_final_params_rf['n_estimators'])
best_final_params_rf['min_samples_split'] = int(best_final_params_rf['min_samples_split'])
best_final_params_rf['min_samples_leaf'] = int(best_final_params_rf['min_samples_leaf'])
# max_depth и max_features уже должны быть в правильном формате из space_eval

# Получаем лучший CV RMSE из trials
try:
    best_cv_rmse_rf = trials_rf.best_trial['result']['loss']
    print(f"\nЛучшие найденные параметры для Random Forest: {best_final_params_rf}")
    print(f"Лучший RMSE на кросс-валидации: {best_cv_rmse_rf:.4f}")
except Exception as e:
     print(f"\nНе удалось извлечь лучший результат из hyperopt trials: {e}")
     print(f"Найденные параметры (могут быть не лучшими): {best_final_params_rf}")
     # Прерываем, если не можем получить лучший результат
     raise SystemExit("Прерывание из-за ошибки в hyperopt.")


# 4. Обучаем финальную модель Random Forest с лучшими найденными параметрами
print("\nОбучение финальной модели Random Forest на всем X_train...")
final_rf_model = RandomForestRegressor(
    **best_final_params_rf,
    random_state=random_seed,
    n_jobs=-1
    # bootstrap=True # Если не было в space
)
# Обучаем на НЕмасштабированных X_train
final_rf_model.fit(X_train, y_train)
print("Финальная модель обучена.")

# 5. Генерация рекурсивного прогноза (используя final_rf_model)
print("\nГенерация рекурсивного прогноза...")
initial_history_rf = y_train.copy()
forecast_rf_recursive = recursive_predict(
    model=final_rf_model, # <<<--- Используем финальную модель RF
    initial_history_series=initial_history_rf,
    n_steps=test_months,
    feature_calculator=calculate_features_for_step,
    feature_order=feature_order,
    scaler=None # RF обучался на немасштабированных признаках
)
# Обновляем словарь прогнозов, сохраняя имя модели
if model_name_rf in recursive_forecasts: del recursive_forecasts[model_name_rf]
recursive_forecasts[model_name_rf] = forecast_rf_recursive
print(f"Прогноз {model_name_rf} сгенерирован и добавлен в словарь.")

# 3. Оценка RMSE
# Проверка длин
if len(forecast_rf_recursive) != len(y_test):
     min_len = min(len(forecast_rf_recursive), len(y_test));
     y_test_eval = y_test[:min_len]; forecast_rf_eval = forecast_rf_recursive[:min_len]
else:
     y_test_eval = y_test; forecast_rf_eval = forecast_rf_recursive

overall_rmse_rf_recursive = np.sqrt(mean_squared_error(y_test_eval, forecast_rf_eval))
print(f"\nОбщий RMSE на тесте для {model_name_rf}: {overall_rmse_rf_recursive:.4f}")

# Добавляем/Обновляем общий RMSE
result_rmse = result_rmse[~result_rmse['method'].isin(['Random Forest', model_name_rf])]
result_rf_rec_entry = {'method': model_name_rf, 'rmse': overall_rmse_rf_recursive}
result_rmse = pd.concat([result_rmse, pd.DataFrame([result_rf_rec_entry])], ignore_index=True)

# Расчет и добавление RMSE по горизонтам
print("RMSE по горизонтам:")
df_horizon_rmse = df_horizon_rmse[~df_horizon_rmse['method'].isin(['Random Forest', model_name_rf])]
horizon_results_list_rf_rec = []
for h in horizons_to_evaluate:
    if h <= len(y_test_eval):
        forecast_h = forecast_rf_eval[:h]
        actual_h = y_test_eval[:h]
        rmse_h = np.sqrt(mean_squared_error(actual_h, forecast_h))
        print(f"  1-{h} мес.: {rmse_h:.4f}")
        horizon_results_list_rf_rec.append({'method': model_name_rf, 'horizon': h, 'rmse': rmse_h})
df_horizon_rmse_rf_rec = pd.DataFrame(horizon_results_list_rf_rec)
df_horizon_rmse = pd.concat([df_horizon_rmse, df_horizon_rmse_rf_rec], ignore_index=True)
print("Результаты RMSE по горизонтам добавлены/обновлены.")

# 4. График прогноза
plt.figure(figsize=(12, 6))
history_to_plot = train_values[train_values.index >= plot_start_date]
plt.plot(history_to_plot.index, history_to_plot, label='Обучающая выборка (часть)')
plt.plot(test.index, test_values, label='Тестовая выборка (реальные)', color='orange', linewidth=2)
forecast_rf_series = pd.Series(forecast_rf_eval, index=test.index[:len(forecast_rf_eval)])
plt.plot(forecast_rf_series.index, forecast_rf_series, label=f'Прогноз {model_name_rf}', color='teal', linestyle='--')
forecast_arima_series = pd.Series(forecast_arima, index=test.index[:len(forecast_arima)])
plt.plot(forecast_arima_series.index, forecast_arima_series, label='Прогноз ARIMA', color='blue', linestyle=':', alpha=0.7)
plt.title(f'Прогноз ИПЦ с помощью {model_name_rf} (Рекурсивный)')
plt.xlabel('Дата'); plt.ylabel('ИПЦ (месячный рост)'); plt.legend(); plt.grid(True)
plt.show()

# 5. График RMSE vs Горизонт (только для этой модели)
plt.figure(figsize=(10, 5))
plt.plot(df_horizon_rmse_rf_rec['horizon'], df_horizon_rmse_rf_rec['rmse'], marker='o', linestyle='-')
plt.title(f'Зависимость RMSE от горизонта прогноза для {model_name_rf}')
plt.xlabel('Горизонт прогноза (месяцы)'); plt.ylabel('RMSE')
plt.xticks(horizons_to_evaluate); plt.grid(True); plt.show()

print(f"--- Анализ {model_name_rf} завершен ---")

# 6. Вывод итоговых таблиц (для сравнения с предыдущими)
print("\nТекущая таблица общих RMSE:")
print(result_rmse.sort_values(by='rmse'))
print("\nТекущая таблица RMSE по горизонтам:")
pivot_table_current = df_horizon_rmse.pivot(index='horizon', columns='method', values='rmse')
current_order = result_rmse.sort_values(by='rmse')['method'].tolist()
# Добавляем и сортируем колонки
for method in current_order:
     if method not in pivot_table_current.columns: pivot_table_current[method] = np.nan
# Убедимся, что новая модель есть в current_order, если вдруг result_rmse не успел обновиться
if model_name_rf not in current_order: current_order.append(model_name_rf)
# Попытаемся вывести в отсортированном порядке, но если колонки нет - выведем как есть
try:
    print(pivot_table_current[current_order].round(4))
except KeyError:
     print("Не удалось отсортировать колонки, вывод как есть:")
     print(pivot_table_current.round(4))

# XGBOOST

In [ ]:
# --- БЛОК 10: Модель XGBoost (Обучение с hyperopt + Рекурсивный Прогноз) ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
# Убираем GridSearchCV, добавляем hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from sklearn.metrics import mean_squared_error
# Убедимся, что tscv определен
# from sklearn.model_selection import TimeSeriesSplit

# --- ПРЕДПОЛАГАЕМ, ЧТО ПЕРЕМЕННЫЕ СУЩЕСТВУЮТ ИЗ ПРЕДЫДУЩИХ БЛОКОВ ---
# X_train, y_train - НЕмасштабированные DataFrame и Series
# X_test, y_test - для теста
# tscv - настроенный TimeSeriesSplit
# calculate_features_for_step, feature_order, test_months, horizons_to_evaluate
# result_rmse, df_horizon_rmse, recursive_forecasts
# plot_start_date, forecast_arima
# train_values, test_values
# random_seed
# recursive_predict
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# --- Дополнительные настройки ---
model_name_xgb = "XGB_Recursive" # <<<--- СОХРАНЯЕМ ИМЯ
MAX_EVALS_HYPEROPT_XGB = 75 # Количество итераций (можно начать с 50-75 и увеличить при необходимости)
# --- Конец Дополнительных настроек ---

print(f"\n--- Обучение и Прогнозирование: {model_name_xgb} ---")

# 1. Определяем пространство поиска для hyperopt XGBoost
space_xgb = {
    'n_estimators': hp.quniform('n_estimators', 100, 400, 25),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'max_depth': hp.quniform('max_depth', 1, 4, 1),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'gamma': hp.uniform('gamma', 0.0, 0.5),
    'reg_alpha': hp.loguniform('reg_alpha', np.log(0.001), np.log(1.0)),
    'reg_lambda': hp.loguniform('reg_lambda', np.log(0.1), np.log(10.0))
}
print(f"Пространство поиска для XGBoost: {space_xgb}")


# 2. Определяем целевую функцию (objective) для hyperopt XGBoost
def objective_xgb(params):
    """
    Целевая функция для hyperopt. Обучает XGBoost с заданными
    параметрами и оценивает его с помощью TimeSeriesSplit кросс-валидации.
    Возвращает средний RMSE по фолдам.
    """
    # Преобразуем типы параметров
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])

    # Создаем модель
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        random_state=random_seed,
        n_jobs=-1, # Используем все ядра
        **params # Распаковываем параметры из hyperopt
    )

    rmses = []
    try:
        # Используем НЕмасштабированные X_train
        for train_idx, val_idx in tscv.split(X_train):
            X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

            model.fit(X_fold_train, y_fold_train) # Можно добавить early_stopping_rounds здесь, если нужно
            preds = model.predict(X_fold_val)

            if not np.all(np.isfinite(preds)) or not np.all(np.isfinite(y_fold_val.values)):
                 continue

            if len(y_fold_val) == 0:
                 continue

            rmse = np.sqrt(mean_squared_error(y_fold_val, preds))
            rmses.append(rmse)

        if not rmses:
            avg_rmse = 1e10
        else:
            avg_rmse = np.mean(rmses)

    except Exception as e:
        # print(f"Error during CV for params={params}: {e}") # Отладка
        avg_rmse = 1e10

    if not np.isfinite(avg_rmse):
        avg_rmse = 1e10

    return {'loss': avg_rmse, 'status': STATUS_OK }

# 3. Запускаем оптимизацию hyperopt
print(f"Запуск hyperopt для подбора параметров XGBoost (max_evals={MAX_EVALS_HYPEROPT_XGB})...")
trials_xgb = Trials()

try:
    rstate_xgb = np.random.default_rng(random_seed)
except AttributeError:
    rstate_xgb = np.random.RandomState(random_seed)

best_params_raw_xgb = fmin(
    fn=objective_xgb,
    space=space_xgb,
    algo=tpe.suggest,
    max_evals=MAX_EVALS_HYPEROPT_XGB,
    trials=trials_xgb,
    rstate=rstate_xgb,
    show_progressbar=True
)

# Преобразуем результат fmin обратно в читаемые параметры
best_final_params_xgb = space_eval(space_xgb, best_params_raw_xgb)

# Преобразуем числовые параметры к нужному типу (int)
best_final_params_xgb['n_estimators'] = int(best_final_params_xgb['n_estimators'])
best_final_params_xgb['max_depth'] = int(best_final_params_xgb['max_depth'])

# Получаем лучший CV RMSE из trials
try:
    best_cv_rmse_xgb = trials_xgb.best_trial['result']['loss']
    print(f"\nЛучшие найденные параметры для XGBoost: {best_final_params_xgb}")
    print(f"Лучший RMSE на кросс-валидации: {best_cv_rmse_xgb:.4f}")
except Exception as e:
     print(f"\nНе удалось извлечь лучший результат из hyperopt trials: {e}")
     print(f"Найденные параметры (могут быть не лучшими): {best_final_params_xgb}")
     raise SystemExit("Прерывание из-за ошибки в hyperopt.")


# 4. Обучаем финальную модель XGBoost с лучшими найденными параметрами
print("\nОбучение финальной модели XGBoost на всем X_train...")
final_xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=random_seed,
    n_jobs=-1,
    **best_final_params_xgb
)
# Обучаем на НЕмасштабированных X_train
final_xgb_model.fit(X_train, y_train)
print("Финальная модель обучена.")

# 5. Генерация рекурсивного прогноза (используя final_xgb_model)
print("\nГенерация рекурсивного прогноза...")
initial_history_xgb = y_train.copy()
forecast_xgb_recursive = recursive_predict(
    model=final_xgb_model, # <<<--- Используем финальную модель XGB
    initial_history_series=initial_history_xgb,
    n_steps=test_months,
    feature_calculator=calculate_features_for_step,
    feature_order=feature_order,
    scaler=None # XGBoost обучался на немасштабированных признаках
)
# Обновляем словарь прогнозов, сохраняя имя модели
if model_name_xgb in recursive_forecasts: del recursive_forecasts[model_name_xgb]
recursive_forecasts[model_name_xgb] = forecast_xgb_recursive
print(f"Прогноз {model_name_xgb} сгенерирован и добавлен в словарь.")

# 3. Оценка RMSE
# Проверка длин
if len(forecast_xgb_recursive) != len(y_test):
     min_len = min(len(forecast_xgb_recursive), len(y_test));
     y_test_eval = y_test[:min_len]; forecast_xgb_eval = forecast_xgb_recursive[:min_len]
else:
     y_test_eval = y_test; forecast_xgb_eval = forecast_xgb_recursive

overall_rmse_xgb_recursive = np.sqrt(mean_squared_error(y_test_eval, forecast_xgb_eval))
print(f"\nОбщий RMSE на тесте для {model_name_xgb}: {overall_rmse_xgb_recursive:.4f}")

# Добавляем/Обновляем общий RMSE
result_rmse = result_rmse[~result_rmse['method'].isin(['XGBoost', model_name_xgb])]
result_xgb_rec_entry = {'method': model_name_xgb, 'rmse': overall_rmse_xgb_recursive}
result_rmse = pd.concat([result_rmse, pd.DataFrame([result_xgb_rec_entry])], ignore_index=True)

# Расчет и добавление RMSE по горизонтам
print("RMSE по горизонтам:")
df_horizon_rmse = df_horizon_rmse[~df_horizon_rmse['method'].isin(['XGBoost', model_name_xgb])]
horizon_results_list_xgb_rec = []
for h in horizons_to_evaluate:
    if h <= len(y_test_eval):
        forecast_h = forecast_xgb_eval[:h]
        actual_h = y_test_eval[:h]
        rmse_h = np.sqrt(mean_squared_error(actual_h, forecast_h))
        print(f"  1-{h} мес.: {rmse_h:.4f}")
        horizon_results_list_xgb_rec.append({'method': model_name_xgb, 'horizon': h, 'rmse': rmse_h})
df_horizon_rmse_xgb_rec = pd.DataFrame(horizon_results_list_xgb_rec)
df_horizon_rmse = pd.concat([df_horizon_rmse, df_horizon_rmse_xgb_rec], ignore_index=True)
print("Результаты RMSE по горизонтам добавлены/обновлены.")

# 4. График прогноза
plt.figure(figsize=(12, 6))
history_to_plot = train_values[train_values.index >= plot_start_date]
plt.plot(history_to_plot.index, history_to_plot, label='Обучающая выборка (часть)')
plt.plot(test.index, test_values, label='Тестовая выборка (реальные)', color='orange', linewidth=2)
forecast_xgb_series = pd.Series(forecast_xgb_eval, index=test.index[:len(forecast_xgb_eval)])
plt.plot(forecast_xgb_series.index, forecast_xgb_series, label=f'Прогноз {model_name_xgb}', color='red', linestyle='--')
forecast_arima_series = pd.Series(forecast_arima, index=test.index[:len(forecast_arima)])
plt.plot(forecast_arima_series.index, forecast_arima_series, label='Прогноз ARIMA', color='blue', linestyle=':', alpha=0.7)
plt.title(f'Прогноз ИПЦ с помощью {model_name_xgb} (Рекурсивный)')
plt.xlabel('Дата'); plt.ylabel('ИПЦ (месячный рост)'); plt.legend(); plt.grid(True)
plt.show()

# 5. График RMSE vs Горизонт (только для этой модели)
plt.figure(figsize=(10, 5))
plt.plot(df_horizon_rmse_xgb_rec['horizon'], df_horizon_rmse_xgb_rec['rmse'], marker='o', linestyle='-')
plt.title(f'Зависимость RMSE от горизонта прогноза для {model_name_xgb}')
plt.xlabel('Горизонт прогноза (месяцы)'); plt.ylabel('RMSE')
plt.xticks(horizons_to_evaluate); plt.grid(True); plt.show()

print(f"--- Анализ {model_name_xgb} завершен ---")

# 6. Вывод итоговых таблиц (для сравнения с предыдущими)
print("\nТекущая таблица общих RMSE:")
print(result_rmse.sort_values(by='rmse'))
print("\nТекущая таблица RMSE по горизонтам:")
pivot_table_current = df_horizon_rmse.pivot(index='horizon', columns='method', values='rmse')
current_order = result_rmse.sort_values(by='rmse')['method'].tolist()
# Добавляем и сортируем колонки
for method in current_order:
     if method not in pivot_table_current.columns: pivot_table_current[method] = np.nan
if model_name_xgb not in current_order: current_order.append(model_name_xgb) # Добавляем новую модель в порядок
try:
    print(pivot_table_current[current_order].round(4))
except KeyError as e:
     print(f"Ошибка при сортировке колонок: {e}. Вывод как есть:")
     print(pivot_table_current.round(4))

In [ ]:
# 4. График прогноза
plt.figure(figsize=(12, 6))
history_to_plot = train_values[train_values.index >= plot_start_date]
plt.plot(history_to_plot.index, history_to_plot, label='Обучающая выборка (часть)')
plt.plot(test.index, test_values, label='Тестовая выборка (реальные)', color='orange', linewidth=2)
forecast_xgb_series = pd.Series(forecast_xgb_eval, index=test.index[:len(forecast_xgb_eval)])
plt.plot(forecast_xgb_series.index, forecast_xgb_series, label=f'Прогноз {model_name_xgb}', color='red', linestyle='--')
forecast_arima_series = pd.Series(forecast_arima, index=test.index[:len(forecast_arima)])
plt.plot(forecast_arima_series.index, forecast_arima_series, label='Прогноз ARIMA', color='blue', linestyle=':', alpha=0.7)
forecast_rf_series = pd.Series(forecast_rf_eval, index=test.index[:len(forecast_rf_eval)])
plt.plot(forecast_rf_series.index, forecast_rf_series, label=f'Прогноз {model_name_rf}', color='teal', linestyle='--')
plt.title(f'Прогноз ИПЦ с помощью RF и XGB (Рекурсивный)')
plt.xlabel('Дата'); plt.ylabel('ИПЦ (месячный рост)'); plt.legend(); plt.grid(True)
plt.show()

# Blend

In [ ]:
# --- БЛОК 11: Бленд ARIMA + Деревья (Обновленный) ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

# --- ПРЕДПОЛАГАЕМ, ЧТО ПЕРЕМЕННЫЕ СУЩЕСТВУЮТ ИЗ ПРЕДЫДУЩИХ БЛОКОВ ---
# recursive_forecasts: Словарь УЖЕ СОДЕРЖИТ 'ARIMA', 'RF_Recursive', 'XGB_Recursive'.
# forecast_arima: Рекурсивный прогноз ARIMA(1,0,0) (numpy array)
# result_rmse: DataFrame с общими RMSE этих моделей.
# df_horizon_rmse: DataFrame с RMSE по горизонтам.
# y_test, test_months, horizons_to_evaluate, history_points_to_show
# y_train, test_ml.index
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# --- 1. Сбор данных для блендинга (ARIMA, RF_R, XGB_R) ---
model_names_for_blend_select = [
    'ARIMA',
    'RF_Recursive',
    'XGB_Recursive'
]
model_name_blend_select = "Blend_ARIMA_Trees" # Имя финального бленда

# Соберем прогнозы из основного словаря recursive_forecasts
try:
    if 'recursive_forecasts' not in locals(): raise NameError("Словарь recursive_forecasts не найден")
    # Заменяем 'ARIMA' в словаре на актуальный прогноз (на всякий случай)
    recursive_forecasts['ARIMA'] = forecast_arima[:test_months]
    missing_forecasts = [name for name in model_names_for_blend_select if name not in recursive_forecasts]
    if missing_forecasts:
        raise KeyError(f"Отсутствуют прогнозы для: {', '.join(missing_forecasts)}")
    # Используем основной словарь recursive_forecasts
    model_forecasts_select = {name: recursive_forecasts[name] for name in model_names_for_blend_select}
except (NameError, KeyError) as e:
    print(f"Ошибка при сборе прогнозов для блендинга: {e}.")
    raise e

# Извлечем ИХ общие RMSE из таблицы 'result_rmse'
try:
    model_rmses_select = result_rmse[result_rmse['method'].isin(model_names_for_blend_select)].set_index('method')['rmse']
    if len(model_rmses_select) != len(model_names_for_blend_select):
         missing_rmse = set(model_names_for_blend_select) - set(model_rmses_select.index)
         raise ValueError(f"Не все RMSE найдены. Отсутствуют: {missing_rmse}")
except Exception as e:
    print(f"Ошибка при извлечении RMSE для блендинга: {e}")
    raise e

print("--- Расчет весов для блендинга (ARIMA + Деревья) ---")
print("RMSE моделей для блендинга:")
print(model_rmses_select)

# Расчет весов
inverse_rmses_select = 1 / model_rmses_select
total_inverse_rmse_select = inverse_rmses_select.sum()
weights_select = inverse_rmses_select / total_inverse_rmse_select
print("\nВеса моделей в бленде:")
print(weights_select.round(4))
print(f"\nСумма весов: {weights_select.sum():.4f}")

# --- 2. Создание комбинированного прогноза (Blend_ARIMA_Trees) ---
print(f"\n--- Создание и оценка {model_name_blend_select} прогноза ---")
forecast_blend_select_recursive = np.zeros(len(y_test), dtype=float)
valid_models_in_blend_select = 0

for method_name, forecast_array in model_forecasts_select.items():
    # Используем forecast_array напрямую из словаря, т.к. обновили 'ARIMA' в recursive_forecasts
    current_forecast = forecast_array

    if len(current_forecast) == len(y_test):
        forecast_blend_select_recursive += current_forecast * weights_select[method_name]
        valid_models_in_blend_select += 1
    else:
        print(f"Предупреждение: Длина прогноза {method_name} ({len(current_forecast)}) не совпадает с y_test ({len(y_test)}).")


if valid_models_in_blend_select != len(model_names_for_blend_select):
    print(f"ВНИМАНИЕ: Не все модели ({valid_models_in_blend_select} из {len(model_names_for_blend_select)}) были использованы в бленде из-за длины.")
    if valid_models_in_blend_select == 0:
        raise ValueError("Невозможно создать бленд.")


recursive_forecasts[model_name_blend_select] = forecast_blend_select_recursive[:test_months]
print(f"Прогноз {model_name_blend_select} сгенерирован и добавлен в словарь.")


# --- 3. Оценка Blend_ARIMA_Trees ---
# Проверка длин
if len(forecast_blend_select_recursive) != len(y_test):
     min_len = min(len(forecast_blend_select_recursive), len(y_test));
     y_test_eval = y_test[:min_len]; forecast_blend_select_eval = forecast_blend_select_recursive[:min_len]
else:
     y_test_eval = y_test; forecast_blend_select_eval = forecast_blend_select_recursive

overall_rmse_blend_select_recursive = np.sqrt(mean_squared_error(y_test_eval, forecast_blend_select_eval))
print(f"Общий RMSE на тесте для {model_name_blend_select}: {overall_rmse_blend_select_recursive:.4f}")

# --- Обновление таблиц результатов ---
# Удаляем старые бленды, если были
result_rmse = result_rmse[~result_rmse['method'].isin(['Blend_ARIMA_Trees', 'Blend_Trees_Recursive', 'Blend_ML_Recursive', 'Blend'])]
df_horizon_rmse = df_horizon_rmse[~df_horizon_rmse['method'].isin(['Blend_ARIMA_Trees', 'Blend_Trees_Recursive', 'Blend_ML_Recursive', 'Blend'])]

# Добавляем новый общий RMSE
result_blend_select_entry = {'method': model_name_blend_select, 'rmse': overall_rmse_blend_select_recursive}
result_rmse = pd.concat([result_rmse, pd.DataFrame([result_blend_select_entry])], ignore_index=True)

print("\nОбновленная таблица общих RMSE:")
print(result_rmse.sort_values(by='rmse'))

# --- Расчет RMSE по горизонтам для Blend_ARIMA_Trees ---
print(f"\n--- Расчет RMSE по горизонтам для {model_name_blend_select} ---")
horizon_results_list_blend_select = []

for h in horizons_to_evaluate:
    if h <= len(y_test_eval):
        forecast_h = forecast_blend_select_eval[:h]
        actual_h = y_test_eval[:h]
        rmse_h = np.sqrt(mean_squared_error(actual_h, forecast_h))
        print(f"RMSE для горизонта 1-{h} мес.: {rmse_h:.4f}")
        horizon_results_list_blend_select.append({'method': model_name_blend_select, 'horizon': h, 'rmse': rmse_h})

df_horizon_rmse_blend_select = pd.DataFrame(horizon_results_list_blend_select)
df_horizon_rmse = pd.concat([df_horizon_rmse, df_horizon_rmse_blend_select], ignore_index=True)

print("\nОбновленная таблица RMSE по горизонтам:")
relevant_methods_select = result_rmse['method'].tolist() # Все текущие модели
pivot_table_select = df_horizon_rmse[df_horizon_rmse['method'].isin(relevant_methods_select)].pivot(index='horizon', columns='method', values='rmse')
for method in relevant_methods_select:
     if method not in pivot_table_select.columns: pivot_table_select[method] = np.nan
current_rmse_order_select = result_rmse.sort_values(by='rmse')['method'].tolist() # Сортируем для вывода
print(pivot_table_select[current_rmse_order_select].round(4))


# --- График прогноза (исправленный) ---
plt.figure(figsize=(12, 6))
# Отображение части обучающей выборки
history_to_plot = train_values[train_values.index >= plot_start_date]
plt.plot(history_to_plot.index, history_to_plot, label='Обучающая выборка (часть)')
# Отображение реальных тестовых данных
plt.plot(test.index, test_values, label='Тестовая выборка (реальные)', color='orange', linewidth=2)

# Отображение прогноза Бленда
forecast_blend_series = pd.Series(forecast_blend_select_eval, index=test.index[:len(forecast_blend_select_eval)])
plt.plot(forecast_blend_series.index, forecast_blend_series, label=f'Прогноз {model_name_blend_select}', color='indigo', linestyle='--')

# Отображение прогноза ARIMA для сравнения
forecast_arima_series = pd.Series(forecast_arima, index=test.index[:len(forecast_arima)])
plt.plot(forecast_arima_series.index, forecast_arima_series, label='Прогноз ARIMA', color='blue', linestyle=':', alpha=0.7)

# Настройки графика
plt.title(f'Прогноз ИПЦ с помощью {model_name_blend_select}')
plt.xlabel('Дата'); plt.ylabel('ИПЦ (месячный рост)'); plt.legend(); plt.grid(True)
plt.show()

# 5. График RMSE vs Горизонт (только для этой модели - Blend_ARIMA_Trees)
# Убедимся, что DataFrame df_horizon_rmse_blend_select существует и не пуст
try:
    if not df_horizon_rmse_blend_select.empty:
        plt.figure(figsize=(10, 5))
        plt.plot(df_horizon_rmse_blend_select['horizon'], df_horizon_rmse_blend_select['rmse'], marker='o', linestyle='-', color='indigo') # Цвет бленда
        plt.title(f'Зависимость RMSE от горизонта прогноза для {model_name_blend_select}')
        plt.xlabel('Горизонт прогноза (месяцы)'); plt.ylabel('RMSE')
        plt.xticks(horizons_to_evaluate); plt.grid(True); plt.show()
    else:
        print(f"Нет данных для построения графика RMSE vs Горизонт для {model_name_blend_select}")
except NameError:
    print(f"DataFrame df_horizon_rmse_blend_select не найден для графика RMSE vs Горизонт.")


print(f"--- Анализ {model_name_blend_select} завершен ---")

# Гибрид Arima + XGBoost

In [ ]:
# --- БЛОК 12: Гибридная Модель ARIMA+XGBoost_на_ошибках (hyperopt для XGB_err) ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA as StatsmodelsARIMA
# Убираем GridSearchCV, добавляем hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from sklearn.metrics import mean_squared_error
# Убедимся, что tscv определен
# from sklearn.model_selection import TimeSeriesSplit

# --- ПРЕДПОЛАГАЕМ, ЧТО ПЕРЕМЕННЫЕ СУЩЕСТВУЮТ ИЗ ПРЕДЫДУЩИХ БЛОКОВ ---
# X_train, y_train - НЕмасштабированные DataFrame и Series
# X_test, y_test - для теста
# tscv - настроенный TimeSeriesSplit
# calculate_features_for_step, feature_order, test_months, horizons_to_evaluate
# result_rmse, df_horizon_rmse, recursive_forecasts
# plot_start_date, forecast_arima - рекурсивный прогноз ARIMA(1,0,0)
# train_values, test_values - для графиков
# random_seed
# recursive_predict
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# --- Дополнительные настройки ---
model_name_hybrid = "ARIMA+XGB_err_Rec_Tuned" # <<<--- СОХРАНЯЕМ ИМЯ
arima_order = (1, 0, 0)
MAX_EVALS_HYPEROPT_XGB_ERR = 50 # Количество итераций для hyperopt XGBoost на ошибках
# --- Конец Дополнительных настроек ---

print(f"\n--- Обучение и Прогнозирование: {model_name_hybrid} ---")

# 1. Подготовка данных для моделирования ошибок ARIMA
print("Подготовка данных для моделирования ошибок ARIMA...")
try:
    # Пересчитываем ошибки ARIMA
    arima_model_full_train = StatsmodelsARIMA(y_train, order=arima_order).fit()
    arima_fitted_values_train = arima_model_full_train.fittedvalues.reindex(y_train.index)
    first_valid_index_arima_fit = arima_fitted_values_train.first_valid_index()
    if first_valid_index_arima_fit is None:
        raise ValueError("ARIMA fitted values не содержат валидных данных.")
    start_pos_arima_fit = y_train.index.get_loc(first_valid_index_arima_fit)

    y_train_aligned_for_errors = y_train.iloc[start_pos_arima_fit:]
    arima_fitted_aligned_for_errors = arima_fitted_values_train.iloc[start_pos_arima_fit:]
    X_train_aligned_for_errors = X_train.reindex(y_train_aligned_for_errors.index)

    if X_train_aligned_for_errors.isnull().values.any():
        print("Предупреждение: NaN в X_train_aligned_for_errors. Удаление строк с NaN...")
        nan_rows_index = X_train_aligned_for_errors[X_train_aligned_for_errors.isnull().any(axis=1)].index
        X_train_aligned_for_errors = X_train_aligned_for_errors.dropna()
        y_train_aligned_for_errors = y_train_aligned_for_errors.drop(nan_rows_index)
        arima_fitted_aligned_for_errors = arima_fitted_aligned_for_errors.drop(nan_rows_index)
        print("Строки с NaN удалены.")

    arima_errors_train_target = y_train_aligned_for_errors - arima_fitted_aligned_for_errors

    print(f"Размер X_train_aligned_for_errors: {X_train_aligned_for_errors.shape}")
    print(f"Размер arima_errors_train_target: {arima_errors_train_target.shape}")
    if X_train_aligned_for_errors.empty or arima_errors_train_target.empty:
         raise ValueError("Недостаточно данных после выравнивания и удаления NaN для обучения XGBoost на ошибках.")

except Exception as e:
    print(f"Ошибка при подготовке данных для ошибок ARIMA: {e}")
    raise e


# 2. Определяем пространство поиска для hyperopt XGBoost на ошибках
space_xgb_errors = {
    'n_estimators': hp.quniform('n_estimators', 50, 300, 25),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'max_depth': hp.quniform('max_depth', 1, 4, 1), # Проверяем неглубокие деревья
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'gamma': hp.uniform('gamma', 0.0, 1.0), # Чуть шире диапазон для gamma
    'reg_alpha': hp.loguniform('reg_alpha', np.log(0.001), np.log(1.0)),
    'reg_lambda': hp.loguniform('reg_lambda', np.log(0.1), np.log(10.0))
}
print(f"Пространство поиска для XGBoost на ошибках: {space_xgb_errors}")

# 3. Определяем целевую функцию (objective) для hyperopt XGBoost на ошибках
def objective_xgb_errors(params):
    """
    Целевая функция для hyperopt. Обучает XGBoost на ОШИБКАХ ARIMA
    и оценивает его с помощью TimeSeriesSplit кросс-валидации.
    Возвращает средний RMSE по фолдам предсказания ошибок.
    """
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])

    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        random_state=random_seed,
        n_jobs=-1,
        **params
    )

    rmses = []
    try:
        # ВАЖНО: CV проводим на выровненных данных и ошибках
        for train_idx, val_idx in tscv.split(X_train_aligned_for_errors):
            # Используем .iloc, так как это Pandas объекты
            X_fold_train_err, X_fold_val_err = X_train_aligned_for_errors.iloc[train_idx], X_train_aligned_for_errors.iloc[val_idx]
            y_fold_train_err, y_fold_val_err = arima_errors_train_target.iloc[train_idx], arima_errors_train_target.iloc[val_idx]

            model.fit(X_fold_train_err, y_fold_train_err)
            preds_err = model.predict(X_fold_val_err)

            if not np.all(np.isfinite(preds_err)) or not np.all(np.isfinite(y_fold_val_err.values)):
                 continue
            if len(y_fold_val_err) == 0:
                 continue

            rmse = np.sqrt(mean_squared_error(y_fold_val_err, preds_err))
            rmses.append(rmse)

        if not rmses: avg_rmse = 1e10
        else: avg_rmse = np.mean(rmses)

    except Exception as e:
        # print(f"Error during CV for errors, params={params}: {e}") # Отладка
        avg_rmse = 1e10

    if not np.isfinite(avg_rmse): avg_rmse = 1e10

    return {'loss': avg_rmse, 'status': STATUS_OK }

# 4. Запускаем оптимизацию hyperopt для XGBoost на ошибках
print(f"Запуск hyperopt для подбора параметров XGBoost на ошибках (max_evals={MAX_EVALS_HYPEROPT_XGB_ERR})...")
trials_xgb_errors = Trials()

try:
    rstate_xgb_err = np.random.default_rng(random_seed)
except AttributeError:
    rstate_xgb_err = np.random.RandomState(random_seed)

best_params_raw_xgb_err = fmin(
    fn=objective_xgb_errors,
    space=space_xgb_errors,
    algo=tpe.suggest,
    max_evals=MAX_EVALS_HYPEROPT_XGB_ERR,
    trials=trials_xgb_errors,
    rstate=rstate_xgb_err,
    show_progressbar=True
)

# Преобразуем и получаем лучшие параметры
best_final_params_xgb_err = space_eval(space_xgb_errors, best_params_raw_xgb_err)
best_final_params_xgb_err['n_estimators'] = int(best_final_params_xgb_err['n_estimators'])
best_final_params_xgb_err['max_depth'] = int(best_final_params_xgb_err['max_depth'])

try:
    best_cv_rmse_xgb_err = trials_xgb_errors.best_trial['result']['loss']
    print(f"\nЛучшие найденные параметры для XGBoost на ошибках: {best_final_params_xgb_err}")
    print(f"Лучший RMSE на CV для ошибок ARIMA: {best_cv_rmse_xgb_err:.4f}")
except Exception as e:
     print(f"\nНе удалось извлечь лучший результат из hyperopt trials для ошибок: {e}")
     raise SystemExit("Прерывание из-за ошибки в hyperopt для ошибок.")

# 5. Обучаем финальную модель XGBoost на ВСЕХ ошибках ARIMA с лучшими параметрами
print("\nОбучение финальной модели XGBoost на всех ошибках ARIMA...")
final_xgb_error_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=random_seed,
    n_jobs=-1,
    **best_final_params_xgb_err # Используем параметры, найденные для ошибок
)
final_xgb_error_model.fit(X_train_aligned_for_errors, arima_errors_train_target)
print("Финальная модель XGBoost на ошибках обучена.")


# 6. Получаем рекурсивный прогноз ARIMA (он уже должен быть в forecast_arima)
test_arima_preds_rec = forecast_arima[:test_months]

# 7. Делаем РЕКУРСИВНЫЙ прогноз ошибок XGBoost с использованием ОТТЮНИНГОВАННОЙ модели
print("\nГенерация рекурсивного прогноза ошибок XGBoost (с оттюнингованной моделью)...")
initial_history_hybrid = y_train.copy() # Используем историю y для расчета признаков!
test_error_preds_xgb_recursive = recursive_predict(
    model=final_xgb_error_model, # <<<--- Используем финальную модель XGBoost на ошибках
    initial_history_series=initial_history_hybrid,
    n_steps=test_months,
    feature_calculator=calculate_features_for_step,
    feature_order=feature_order,
    scaler=None # XGBoost обучался на немасштабированных признаках
)
print("Рекурсивный прогноз ошибок сгенерирован.")

# 8. Комбинируем рекурсивные прогнозы
print("\nКомбинирование прогнозов ARIMA и XGBoost_error...")
final_forecast_hybrid_recursive = test_arima_preds_rec + test_error_preds_xgb_recursive
# Обновляем словарь прогнозов, СОХРАНЯЯ ИМЯ МОДЕЛИ
if model_name_hybrid in recursive_forecasts: del recursive_forecasts[model_name_hybrid]
recursive_forecasts[model_name_hybrid] = final_forecast_hybrid_recursive[:test_months]
print(f"Прогноз {model_name_hybrid} сгенерирован и добавлен в словарь.")

# 6. Оценка RMSE
# Проверка длин (на всякий случай, если recursive_predict вернет меньше шагов)
if len(final_forecast_hybrid_recursive) != len(y_test):
     min_len = min(len(final_forecast_hybrid_recursive), len(y_test))
     print(f"Предупреждение: Длина гибридного прогноза ({len(final_forecast_hybrid_recursive)}) не совпадает с y_test ({len(y_test)}). Используется длина {min_len}.")
     y_test_eval = y_test[:min_len]
     forecast_hybrid_eval = final_forecast_hybrid_recursive[:min_len]
else:
     y_test_eval = y_test
     forecast_hybrid_eval = final_forecast_hybrid_recursive

overall_rmse_hybrid_recursive = np.sqrt(mean_squared_error(y_test_eval, forecast_hybrid_eval))
print(f"\nОбщий RMSE на тесте для {model_name_hybrid}: {overall_rmse_hybrid_recursive:.4f}")

# Добавляем/Обновляем общий RMSE
# Удаляем старые гибриды или гибриды с похожими именами
result_rmse = result_rmse[~result_rmse['method'].str.contains('ARIMA\+XGB_err', case=False)]
result_hybrid_entry = {'method': model_name_hybrid, 'rmse': overall_rmse_hybrid_recursive}
result_rmse = pd.concat([result_rmse, pd.DataFrame([result_hybrid_entry])], ignore_index=True)

# Расчет и добавление RMSE по горизонтам
print("RMSE по горизонтам:")
df_horizon_rmse = df_horizon_rmse[~df_horizon_rmse['method'].str.contains('ARIMA\+XGB_err', case=False)]
horizon_results_list_hybrid_rec = []
for h in horizons_to_evaluate:
    if h <= len(forecast_hybrid_eval): # Используем длину фактического прогноза
        forecast_h_step = forecast_hybrid_eval[:h]
        actual_h_step = y_test_eval[:h]
        rmse_h = np.sqrt(mean_squared_error(actual_h_step, forecast_h_step))
        print(f"  1-{h} мес.: {rmse_h:.4f}")
        horizon_results_list_hybrid_rec.append({'method': model_name_hybrid, 'horizon': h, 'rmse': rmse_h})
df_horizon_rmse_hybrid_rec = pd.DataFrame(horizon_results_list_hybrid_rec)
df_horizon_rmse = pd.concat([df_horizon_rmse, df_horizon_rmse_hybrid_rec], ignore_index=True)
print("Результаты RMSE по горизонтам добавлены/обновлены.")

# 7. График прогноза
plt.figure(figsize=(12, 6))
history_to_plot_hybrid = train_values[train_values.index >= plot_start_date] # Используем train_values
plt.plot(history_to_plot_hybrid.index, history_to_plot_hybrid, label='Обучающая выборка (часть)')
plt.plot(y_test_eval.index, y_test_eval, label='Тестовая выборка (реальные)', color='orange', linewidth=2) # Используем y_test_eval
forecast_hybrid_series = pd.Series(forecast_hybrid_eval, index=y_test_eval.index[:len(forecast_hybrid_eval)])
plt.plot(forecast_hybrid_series.index, forecast_hybrid_series, label=f'Прогноз {model_name_hybrid}', color='brown', linestyle='--')
# Прогноз ARIMA для сравнения
forecast_arima_series_plot = pd.Series(test_arima_preds_rec, index=y_test_eval.index[:len(test_arima_preds_rec)])
plt.plot(forecast_arima_series_plot.index, forecast_arima_series_plot, label='Прогноз ARIMA (база)', color='blue', linestyle=':', alpha=0.7)
plt.title(f'Прогноз ИПЦ с помощью {model_name_hybrid}')
plt.xlabel('Дата'); plt.ylabel('ИПЦ (месячный рост)'); plt.legend(); plt.grid(True)
plt.show()

# 8. График RMSE vs Горизонт (только для этой модели)
if not df_horizon_rmse_hybrid_rec.empty:
    plt.figure(figsize=(10, 5))
    plt.plot(df_horizon_rmse_hybrid_rec['horizon'], df_horizon_rmse_hybrid_rec['rmse'], marker='o', linestyle='-')
    plt.title(f'Зависимость RMSE от горизонта прогноза для {model_name_hybrid}')
    plt.xlabel('Горизонт прогноза (месяцы)'); plt.ylabel('RMSE')
    plt.xticks(horizons_to_evaluate); plt.grid(True); plt.show()
else:
    print(f"Нет данных для построения графика RMSE vs Горизонт для {model_name_hybrid}")


print(f"--- Анализ {model_name_hybrid} завершен ---")

# 9. Вывод итоговых таблиц
print("\nТекущая таблица общих RMSE:")
print(result_rmse.sort_values(by='rmse'))
print("\nТекущая таблица RMSE по горизонтам:")
pivot_table_current_hybrid = df_horizon_rmse.pivot(index='horizon', columns='method', values='rmse')
current_order_hybrid = result_rmse.sort_values(by='rmse')['method'].tolist()

# Добавляем недостающие колонки, если они есть в current_order_hybrid, но нет в pivot_table_current_hybrid
for method in current_order_hybrid:
     if method not in pivot_table_current_hybrid.columns:
         pivot_table_current_hybrid[method] = np.nan
# Убедимся, что новая модель есть в current_order_hybrid, если result_rmse не успел обновиться корректно
if model_name_hybrid not in current_order_hybrid:
    current_order_hybrid.append(model_name_hybrid)
# Попытаемся вывести в отсортированном порядке, но если колонки нет - выведем как есть
try:
    print(pivot_table_current_hybrid[current_order_hybrid].round(4))
except KeyError as e:
     print(f"Ошибка при сортировке колонок для вывода RMSE по горизонтам: {e}. Вывод как есть:")
     print(pivot_table_current_hybrid.round(4))

In [ ]:
# 7. График прогноза
plt.figure(figsize=(12, 6))
history_to_plot_hybrid = train_values[train_values.index >= plot_start_date] # Используем train_values
plt.plot(history_to_plot_hybrid.index, history_to_plot_hybrid, label='Обучающая выборка (часть)')
plt.plot(y_test_eval.index, y_test_eval, label='Тестовая выборка (реальные)', color='orange', linewidth=2) # Используем y_test_eval
forecast_hybrid_series = pd.Series(forecast_hybrid_eval, index=y_test_eval.index[:len(forecast_hybrid_eval)])
plt.plot(forecast_hybrid_series.index, forecast_hybrid_series, label=f'Прогноз {model_name_hybrid}', color='brown', linestyle='--')
forecast_blend_series = pd.Series(forecast_blend_select_eval, index=test.index[:len(forecast_blend_select_eval)])
plt.plot(forecast_blend_series.index, forecast_blend_series, label=f'Прогноз {model_name_blend_select}', color='indigo', linestyle='--')
# Прогноз ARIMA для сравнения
forecast_arima_series_plot = pd.Series(test_arima_preds_rec, index=y_test_eval.index[:len(test_arima_preds_rec)])
plt.plot(forecast_arima_series_plot.index, forecast_arima_series_plot, label='Прогноз ARIMA (база)', color='blue', linestyle=':', alpha=0.7)
plt.title(f'Прогноз ИПЦ с помощью {model_name_hybrid}')
plt.xlabel('Дата'); plt.ylabel('ИПЦ (месячный рост)'); plt.legend(); plt.grid(True)
plt.show()

# LSTM

In [ ]:
# --- БЛОК 11: Модель LSTM (Обучение + Рекурсивный Прогноз) ---

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import tensorflow as tf

# --- ПРЕДПОЛАГАЕМ, ЧТО ПЕРЕМЕННЫЕ СУЩЕСТВУЮТ ИЗ ПРЕДЫДУЩИХ БЛОКОВ ---
# data_ml: DataFrame с колонкой 't' и DatetimeIndex
# train_ml: DataFrame обучающей выборки (для определения длины)
# test: DataFrame тестовой выборки (для индекса графика)
# y_test: pd.Series с реальными значениями на тесте
# test_months: Количество месяцев в тесте (24)
# horizons_to_evaluate: Список горизонтов [1, 2, 3, 6, 12, 18, 24]
# result_rmse, df_horizon_rmse: DataFrames для результатов
# recursive_forecasts: Словарь для прогнозов
# plot_start_date, history_points_to_show
# forecast_nonseasonal (или forecast_arima): Прогноз ARIMA
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# --- Настройки LSTM ---
cpi_value_column_lstm = 't' # Используем колонку 't' из data_ml
look_back_lstm = 12
lstm_units = 50
epochs = 100
batch_size = 16
validation_split = 0.1
patience_early_stopping = 10
model_name_lstm = "LSTM"
random_seed = 42
tf.random.set_seed(random_seed)
np.random.seed(random_seed) # Для numpy операций, если они есть в подготовке
# --- Конец Настроек ---

print(f"\n--- Обучение и Прогнозирование: {model_name_lstm} ---")

# 1. Подготовка данных (масштабирование и создание последовательностей)
print("Подготовка данных для LSTM...")
lstm_input_series = data_ml[cpi_value_column_lstm].copy()
scaler_lstm = MinMaxScaler(feature_range=(0, 1))
scaled_data_lstm = scaler_lstm.fit_transform(lstm_input_series.values.reshape(-1, 1))

def create_sequences(data, look_back=1):
    X, Y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), 0])
        Y.append(data[i + look_back, 0])
    return np.array(X), np.array(Y)

X_seq_lstm, y_seq_lstm = create_sequences(scaled_data_lstm, look_back_lstm)

# Разделение на train/test
n_train_sequences_lstm = len(train_ml) - look_back_lstm
X_train_lstm = X_seq_lstm[:n_train_sequences_lstm]
y_train_lstm = y_seq_lstm[:n_train_sequences_lstm]
# X_test_lstm_input нам не нужен для рекурсивного прогноза

# Reshape X_train_lstm для обучения
X_train_lstm = np.reshape(X_train_lstm, (X_train_lstm.shape[0], X_train_lstm.shape[1], 1))
print(f"Размер обучающей выборки LSTM (X, y): {X_train_lstm.shape}, {y_train_lstm.shape}")

# 2. Построение и компиляция модели
print("\nПостроение и компиляция модели LSTM...")
model_lstm = Sequential([
    LSTM(units=lstm_units, input_shape=(look_back_lstm, 1)),
    Dense(units=1)
])
model_lstm.compile(optimizer='adam', loss='mean_squared_error')
# model_lstm.summary() # Можно раскомментировать

# 3. Обучение модели
print("\nОбучение модели LSTM...")
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_early_stopping,
                               verbose=0, mode='min', restore_best_weights=True) # verbose=0 тихий режим
history = model_lstm.fit(X_train_lstm, y_train_lstm,
                         epochs=epochs, batch_size=batch_size,
                         validation_split=validation_split,
                         callbacks=[early_stopping], verbose=0)
print(f"Обучение завершено. Остановлено на эпохе: {early_stopping.stopped_epoch+1}") # +1 т.к. нумерация с 0
print(f"Лучшая val_loss: {early_stopping.best:.4f}")

# 4. Генерация рекурсивного прогноза
print("\nГенерация рекурсивного прогноза...")
last_train_sequence_scaled = X_train_lstm[-1] # Берем последнюю ПОСЛЕДОВАТЕЛЬНОСТЬ из X_train_lstm
predictions_scaled_lstm = []
current_sequence_lstm = last_train_sequence_scaled.reshape(1, look_back_lstm, 1)

for i in range(test_months):
    next_pred_scaled = model_lstm.predict(current_sequence_lstm, verbose=0)[0, 0]
    predictions_scaled_lstm.append(next_pred_scaled)
    next_sequence_values = np.append(current_sequence_lstm[0, 1:, 0], next_pred_scaled)
    current_sequence_lstm = next_sequence_values.reshape(1, look_back_lstm, 1)

predictions_scaled_lstm = np.array(predictions_scaled_lstm)
forecast_lstm = scaler_lstm.inverse_transform(predictions_scaled_lstm.reshape(-1, 1)).flatten()
# Добавляем прогноз в общий словарь
recursive_forecasts[model_name_lstm] = forecast_lstm[:test_months]
print(f"Прогноз {model_name_lstm} сгенерирован и добавлен в словарь.")

# 5. Оценка RMSE
# Проверка длин
if len(forecast_lstm) != len(y_test):
     min_len = min(len(forecast_lstm), len(y_test));
     y_test_eval = y_test[:min_len]; forecast_lstm_eval = forecast_lstm[:min_len]
else:
     y_test_eval = y_test; forecast_lstm_eval = forecast_lstm

overall_rmse_lstm = np.sqrt(mean_squared_error(y_test_eval, forecast_lstm_eval))
print(f"\nОбщий RMSE на тесте для {model_name_lstm}: {overall_rmse_lstm:.4f}")

# Добавляем/Обновляем общий RMSE
result_rmse = result_rmse[~result_rmse['method'].isin([model_name_lstm])]
result_lstm_entry = {'method': model_name_lstm, 'rmse': overall_rmse_lstm}
result_rmse = pd.concat([result_rmse, pd.DataFrame([result_lstm_entry])], ignore_index=True)

# Расчет и добавление RMSE по горизонтам
print("RMSE по горизонтам:")
df_horizon_rmse = df_horizon_rmse[~df_horizon_rmse['method'].isin([model_name_lstm])]
horizon_results_list_lstm = []
for h in horizons_to_evaluate:
    if h <= len(y_test_eval):
        forecast_h = forecast_lstm_eval[:h]
        actual_h = y_test_eval[:h]
        rmse_h = np.sqrt(mean_squared_error(actual_h, forecast_h))
        print(f"  1-{h} мес.: {rmse_h:.4f}") # Вывод промежуточного RMSE
        horizon_results_list_lstm.append({'method': model_name_lstm, 'horizon': h, 'rmse': rmse_h})
df_horizon_rmse_lstm = pd.DataFrame(horizon_results_list_lstm)
df_horizon_rmse = pd.concat([df_horizon_rmse, df_horizon_rmse_lstm], ignore_index=True)
print("Результаты RMSE по горизонтам добавлены/обновлены.")

# 6. График прогноза
plt.figure(figsize=(12, 6))
history_to_plot = train_values[train_values.index >= plot_start_date] # Используем train_values
plt.plot(history_to_plot.index, history_to_plot, label='Обучающая выборка (часть)')
plt.plot(test.index, test_values, label='Тестовая выборка (реальные)', color='orange', linewidth=2) # Используем test_values
forecast_lstm_series = pd.Series(forecast_lstm_eval, index=test.index[:len(forecast_lstm_eval)])
plt.plot(forecast_lstm_series.index, forecast_lstm_series, label=f'Прогноз {model_name_lstm}', color='lime', linestyle='--')
# Добавляем ARIMA для сравнения (используем forecast_arima)
forecast_arima_series = pd.Series(forecast_arima, index=test.index[:len(forecast_arima)])
plt.plot(forecast_arima_series.index, forecast_arima_series, label='Прогноз ARIMA', color='blue', linestyle=':', alpha=0.7)
plt.title(f'Прогноз ИПЦ с помощью {model_name_lstm}')
plt.xlabel('Дата'); plt.ylabel('ИПЦ (месячный рост)'); plt.legend(); plt.grid(True)
plt.show()

# 7. График RMSE vs Горизонт (только для этой модели)
plt.figure(figsize=(10, 5))
plt.plot(df_horizon_rmse_lstm['horizon'], df_horizon_rmse_lstm['rmse'], marker='o', linestyle='-')
plt.title(f'Зависимость RMSE от горизонта прогноза для {model_name_lstm}')
plt.xlabel('Горизонт прогноза (месяцы)'); plt.ylabel('RMSE')
plt.xticks(horizons_to_evaluate); plt.grid(True); plt.show()

print(f"--- Анализ {model_name_lstm} завершен ---")

# 8. Вывод итоговых таблиц
print("\nТекущая таблица общих RMSE:")
print(result_rmse.sort_values(by='rmse'))
print("\nТекущая таблица RMSE по горизонтам:")
pivot_table_current = df_horizon_rmse.pivot(index='horizon', columns='method', values='rmse')
current_order = result_rmse.sort_values(by='rmse')['method'].tolist()
for method in current_order:
     if method not in pivot_table_current.columns: pivot_table_current[method] = np.nan
if model_name_lstm not in current_order: current_order.append(model_name_lstm)
try:
    print(pivot_table_current[current_order].round(4))
except KeyError as e:
     print(f"Ошибка при сортировке колонок: {e}. Вывод как есть:")
     print(pivot_table_current.round(4))

In [ ]:
print(pivot_table_current)

# Графики

In [ ]:
# --- БЛОК 13: Финальные Сравнительные Графики (Рекурсивные прогнозы) ---

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# --- ПРЕДПОЛАГАЕМ, ЧТО ПЕРЕМЕННЫЕ СУЩЕСТВУЮТ ---
# df_horizon_rmse: Актуальный DataFrame с RMSE по горизонтам для всех рекурсивных моделей
# result_rmse: Актуальный DataFrame с общими RMSE
# recursive_forecasts: Актуальный словарь со всеми прогнозами
# y_train, y_test: Series с обучающими и тестовыми данными
# train_values, test_values: То же самое
# test_ml.index: Индекс тестового периода
# plot_start_date, history_points_to_show, horizons_to_evaluate
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

print("\n--- Построение Финальных Сравнительных Графиков ---")

# 1. График RMSE vs Горизонт
plt.figure(figsize=(14, 8))

# Определяем порядок моделей по финальному общему RMSE
final_model_order = result_rmse.sort_values(by='rmse')['method'].tolist()

# Задаем палитру
custom_palette = {
    "ARIMA": "blue", "Mean": "grey", "LSTM": "lime", "Median": "black",
    "Blend_ARIMA_Trees": "indigo", "XGB_Recursive": "red", "ARIMA+XGB_err_Rec_Tuned":"darkred",
    "RF_Recursive": "teal", "Lasso_Recursive": "purple", "Ridge_Recursive": "orange",
    "Trend": "pink",
    "Blend_ML_Recursive": "fuchsia", "Blend_Trees_Recursive": "cyan",
    "ARIMA+XGB_err_Rec": "brown"
}

# Строим график
sns.lineplot(data=df_horizon_rmse, x='horizon', y='rmse', hue='method',
             hue_order=final_model_order, palette=custom_palette, marker='o', linewidth=1.5)

plt.title('Сравнение RMSE моделей на разных горизонтах прогноза (Рекурсивные)')
plt.xlabel('Горизонт прогноза (месяцы)'); plt.ylabel('RMSE')
plt.xticks(horizons_to_evaluate); plt.grid(True)
plt.legend(title='Модель', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout(rect=[0, 0, 0.85, 1]); plt.show()


# 2. График Реальность vs Прогнозы
plt.figure(figsize=(15, 8))
# История
history_to_plot = train_values[train_values.index >= '2022-09-01']
plt.plot(history_to_plot.index, history_to_plot, label='Обуч. (часть)')
# Реальность
plt.plot(test.index, test_values, label='Реальные (тест)', color='orange', linewidth=2.5, marker='o', markersize=4, zorder=10)

# Выбираем КЛЮЧЕВЫЕ модели для отображения (лидеры и лучшие ML/ансамбли)
# models_to_plot_final = ['ARIMA', 'LSTM', 'Mean', 'Median', 'Blend_ARIMA_Trees', 'XGB_Recursive', 'ARIMA+XGB_err_Rec', 'RF_Recursive', 'Trend']

for model_name in final_model_order:
    if model_name in recursive_forecasts:
        forecast = recursive_forecasts[model_name]
        plot_len = min(len(forecast), len(test.index)) # Используем test.index
        # Создаем Series для корректной отрисовки с DatetimeIndex
        forecast_series = pd.Series(forecast[:plot_len], index=test.index[:plot_len])
        plt.plot(forecast_series.index, forecast_series,
                 label=f'Прогноз {model_name}',
                 color=custom_palette.get(model_name, None),
                 linestyle='--', linewidth=1.5, alpha=0.9)
    else:
        print(f"Предупреждение: Прогноз для {model_name} не найден в словаре recursive_forecasts для графика.")

plt.title('Сравнение прогнозов ключевых моделей с реальными данными (Рекурсивные)')
plt.xlabel('Дата'); plt.ylabel('ИПЦ (месячный рост)')
plt.legend(loc='upper left', bbox_to_anchor=(1.02, 1))
plt.grid(True); plt.tight_layout(rect=[0, 0, 0.85, 1]); plt.show()

# DM-test

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import t
import collections

def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Проверка длин
    if not (len(actual_lst) == len(pred1_lst) == len(pred2_lst)):
         raise ValueError("Lengths of actual_lst, pred1_lst and pred2_lst must be equal.")
    if len(actual_lst) < 2:
         return collections.namedtuple('dm_return' , 'DM p_value')(DM = np.nan, p_value = np.nan)
    if h < 1:
         raise ValueError("Horizon h must be 1 or greater.")

    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []

    # Calculate loss differential
    if crit == "MSE":
        for actual,p1,p2 in zip(actual_lst, pred1_lst, pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif crit == "MAD":
        for actual,p1,p2 in zip(actual_lst, pred1_lst, pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    else:
         raise ValueError(f"Criterion {crit} not supported. Use 'MSE' or 'MAD'.")

    # Mean of d
    mean_d = pd.Series(d_lst).mean()
    if np.isnan(mean_d):
        return collections.namedtuple('dm_return' , 'DM p_value')(DM = np.nan, p_value = np.nan)

    # Newey-West variance estimator
    T = float(len(d_lst))
    max_lag = min(h - 1, T - 1)

    gamma = []
    for lag in range(int(max_lag + 1)):
        autoCov = 0
        for i in range(lag, len(d_lst)):
              autoCov += (d_lst[i] - mean_d) * (d_lst[i-lag] - mean_d)
        gamma.append(autoCov / T)

    V_d = gamma[0]
    if max_lag > 0:
        for k in range(1, int(max_lag + 1)):
            weight = 1 - (k / (max_lag + 1))
            V_d += 2 * weight * gamma[k]

    # DM statistic
    if V_d <= 1e-9:
        return collections.namedtuple('dm_return' , 'DM p_value')(DM = np.nan, p_value = 1.0)

    DM_stat = mean_d / np.sqrt(V_d / T)
    df = T - 1
    if df <= 0:
        return collections.namedtuple('dm_return' , 'DM p_value')(DM = np.nan, p_value = np.nan)

    p_value = 2 * t.cdf(-abs(DM_stat), df = df)
    rt = collections.namedtuple('dm_return' , 'DM p_value')(DM = DM_stat, p_value = p_value)
    return rt

In [ ]:
# --- БЛОК 5: Финальный Тест Диболда-Мариано (Метод Холма) ---

from statsmodels.sandbox.stats.multicomp import multipletests
from itertools import combinations
import pandas as pd # Убедимся, что pandas импортирован
import numpy as np # И numpy

# --- ПРЕДПОЛАГАЕМ, ЧТО ПЕРЕМЕННЫЕ СУЩЕСТВУЮТ ИЗ ПРЕДЫДУЩИХ БЛОКОВ ---
# recursive_forecasts: Словарь со всеми прогнозами, включая ключевые модели.
# df_horizon_rmse: DataFrame с RMSE по горизонтам для всех моделей.
# y_test: Series с реальными значениями на тесте.
# horizons_to_evaluate: Список горизонтов [1, 2, 3, 6, 12, 18, 24].
# dm_test: Функция теста Диболда-Мариано (должна быть определена ранее).
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---


print("\n--- Финальный Тест Диболда-Мариано (Метод Холма) ---")
alpha_family = 0.05
# Выбираем ключевых игроков для сравнения
models_to_compare_dm = ['ARIMA', 'LSTM', 'Mean', 'Blend_ARIMA_Trees', 'XGB_Recursive', 'ARIMA+XGB_err_Rec_Tuned', 'RF_Recursive'] # Выбрали лучших и базовые
# models_to_compare_dm = final_model_order
# Убедимся, что все модели из списка есть в наших результатах
existing_models_for_dm = [m for m in models_to_compare_dm if m in result_rmse['method'].values]
if len(existing_models_for_dm) < len(models_to_compare_dm):
    print("Предупреждение: Не все модели из списка models_to_compare_dm найдены в результатах.")
    print("Сравнение будет проведено для:", existing_models_for_dm)
models_to_compare_dm = existing_models_for_dm # Используем только существующие

# Пересчитываем поправку для этого набора сравнений
num_models_dm = len(models_to_compare_dm)
num_pairs_dm = num_models_dm * (num_models_dm - 1) // 2
valid_horizons_for_dm = [h for h in horizons_to_evaluate if h >= 2 and h <= len(y_test)]
num_valid_horizons_for_dm = len(valid_horizons_for_dm)
total_tests_dm = num_pairs_dm * num_valid_horizons_for_dm
# alpha_individual_holm = alpha_family # Для метода Холма НЕ нужен отдельный порог, используем общий alpha

print(f"Сравнение {num_models_dm} моделей на {num_valid_horizons_for_dm} горизонтах ({num_pairs_dm} пар на горизонт, {total_tests_dm} тестов)")
print(f"Контроль FWER на уровне {alpha_family:.2f} для каждого горизонта методом Холма.")

dm_results_final = {} # Словарь для p-values этого набора тестов

for h in valid_horizons_for_dm:
    print(f"\n--- Горизонт: 1-{h} мес. ---")
    actual_h = y_test[:h]
    p_values_h = []
    pairs_h = []

    # Используем combinations из itertools
    for model1, model2 in combinations(models_to_compare_dm, 2):
        # Проверяем наличие прогнозов в основном словаре
        if model1 not in recursive_forecasts or model2 not in recursive_forecasts:
             print(f"    Предупреждение: Пропуск пары {model1} vs {model2} - прогноз отсутствует.")
             continue
        forecast1_h = recursive_forecasts[model1][:h]
        forecast2_h = recursive_forecasts[model2][:h]

        # Проверка совпадения длин перед тестом
        if len(actual_h) != len(forecast1_h) or len(actual_h) != len(forecast2_h):
            print(f"    Предупреждение: Несовпадение длин для {model1} vs {model2} на горизонте {h}. Пропуск.")
            continue

        try:
            dm_res = dm_test(actual_h, forecast1_h, forecast2_h, h=h, crit="MSE")
            p_val = dm_res.p_value
            if not pd.isna(p_val):
                 p_values_h.append(p_val)
                 pairs_h.append(tuple(sorted((model1, model2)))) # Сохраняем пару
        except Exception as e:
            print(f"    Ошибка DM теста для {model1} vs {model2} на горизонте {h}: {e}")

    if not p_values_h:
        print("\nПары со значимыми различиями (Метод Холма):")
        print("  Нет (нет валидных p-values).")
        continue

    # Применяем метод Холма
    try:
        reject_holm, pvals_corrected_holm, _, _ = multipletests(p_values_h,
                                                                 alpha=alpha_family,
                                                                 method='holm')
    except Exception as e:
        print(f"    Ошибка при применении multipletests (Holm): {e}")
        continue


    print("\nПары со значимыми различиями (Метод Холма):")
    significant_pairs_holm = []
    for i, reject in enumerate(reject_holm):
        if reject:
            model1, model2 = pairs_h[i] # Получаем пару по индексу
            p_val_original = p_values_h[i]
            p_val_corrected = pvals_corrected_holm[i]
            try:
                # Получаем RMSE из df_horizon_rmse
                rmse1 = df_horizon_rmse.loc[(df_horizon_rmse['method'] == model1) & (df_horizon_rmse['horizon'] == h), 'rmse'].iloc[0]
                rmse2 = df_horizon_rmse.loc[(df_horizon_rmse['method'] == model2) & (df_horizon_rmse['horizon'] == h), 'rmse'].iloc[0]
                better_model = model1 if rmse1 < rmse2 else model2
                worse_model = model2 if rmse1 < rmse2 else model1
                significant_pairs_holm.append(f"  {better_model} лучше {worse_model} (p_orig={p_val_original:.4f}, p_holm={p_val_corrected:.4f})")
            except IndexError:
                 significant_pairs_holm.append(f"  {model1} vs {model2} (p_orig={p_val_original:.4f}, p_holm={p_val_corrected:.4f}) - не удалось сравнить RMSE")
            except Exception as e_inner:
                 print(f"    Внутренняя ошибка при сравнении RMSE для {model1} vs {model2}: {e_inner}")
                 significant_pairs_holm.append(f"  {model1} vs {model2} (p_orig={p_val_original:.4f}, p_holm={p_val_corrected:.4f}) - ошибка RMSE")


    if significant_pairs_holm:
        significant_pairs_holm.sort()
        for pair in significant_pairs_holm: print(pair)
    else:
        print("  Нет.")

print("\n--- Тест Диболда-Мариано завершен ---")

In [ ]:
# --- ЯЧЕЙКА ДЛЯ СОХРАНЕНИЯ ПРОГНОЗОВ В EXCEL ---

import pandas as pd

# 1. Определяем, какие прогнозы моделей мы хотим сохранить
#    Возьмем тот же список, что и для DM-теста, или любой другой нужный набор.
models_to_save = ['ARIMA', 'LSTM', 'Mean', 'Median', 'Trend', # Добавим все бенчмарки
                  'RF_Recursive', 'XGB_Recursive',
                  'Lasso_Recursive', 'Ridge_Recursive', # Линейные ML
                  'Blend_ARIMA_Trees',
                  'ARIMA+XGB_err_Rec_Tuned'] # Твой лучший гибрид
                  # Добавь сюда 'ARIMA+XGB_err_Rec', если хочешь сохранить и его для сравнения

# 2. Убедимся, что y_test (реальные значения) доступен и имеет правильный индекс
#    Предполагаем, что y_test - это pd.Series с DatetimeIndex тестового периода
if 'y_test' not in locals() or not isinstance(y_test, pd.Series):
    print("Ошибка: Переменная y_test (реальные значения) не найдена или имеет неверный тип.")
    # Можно попытаться ее восстановить, если есть test_values и test.index
    if 'test_values' in locals() and 'test' in locals() and hasattr(test, 'index'):
        try:
            y_test_restored = pd.Series(test_values, index=test.index[:len(test_values)])
            if not y_test_restored.empty:
                 y_test = y_test_restored # Восстанавливаем y_test
                 print("Переменная y_test восстановлена.")
            else:
                 raise ValueError("Не удалось восстановить y_test.")
        except Exception as e:
            print(f"Не удалось восстановить y_test: {e}")
            # Здесь можно либо прервать выполнение, либо продолжить без y_test
            # Для сохранения только прогнозов можно и без y_test, но с ним полезнее
else:
    print("Переменная y_test найдена.")


# 3. Создаем DataFrame для сохранения
#    Индекс будет датами тестового периода
try:
    # Попытаемся использовать индекс из y_test, если он есть и корректен
    if 'y_test' in locals() and not y_test.empty:
        forecast_dates_index = y_test.index[:test_months] # test_months должно быть определено
    # Иначе, если y_test нет, но есть test_ml (из блока подготовки ML данных)
    elif 'test_ml' in locals() and not test_ml.empty:
        forecast_dates_index = test_ml.index[:test_months]
    # Иначе, если есть просто test (из самого начала)
    elif 'test' in locals() and not test.empty:
         forecast_dates_index = test.index[:test_months]
    else:
        raise ValueError("Не удалось определить индекс для дат прогнозов. Убедитесь, что y_test, test_ml или test существуют.")

    df_all_forecasts = pd.DataFrame(index=forecast_dates_index)

    # Добавляем реальные значения, если y_test существует
    if 'y_test' in locals() and not y_test.empty:
        df_all_forecasts['Actual'] = y_test.values[:len(df_all_forecasts)] # Убедимся, что длина совпадает
    else:
        print("Предупреждение: Реальные значения y_test не будут сохранены.")

except Exception as e:
    print(f"Ошибка при создании DataFrame для сохранения: {e}")
    # Прерываем, так как без индекса или y_test сохранение будет неполноценным
    raise SystemExit("Прерывание из-за ошибки подготовки DataFrame для сохранения.")


# 4. Добавляем прогнозы каждой модели в DataFrame
#    Предполагаем, что `recursive_forecasts` - это словарь, где ключ - имя модели, значение - numpy array прогноза
if 'recursive_forecasts' not in locals():
    raise NameError("Словарь recursive_forecasts не найден!")

for model_name in models_to_save:
    if model_name in recursive_forecasts:
        forecast_values = recursive_forecasts[model_name]
        # Обрезаем или дополняем NaN, если длина прогноза не совпадает с длиной индекса
        if len(forecast_values) == len(df_all_forecasts):
            df_all_forecasts[model_name] = forecast_values
        elif len(forecast_values) > len(df_all_forecasts):
            df_all_forecasts[model_name] = forecast_values[:len(df_all_forecasts)]
            print(f"Предупреждение: Прогноз для {model_name} был длиннее тестового периода и был обрезан.")
        else: # len(forecast_values) < len(df_all_forecasts)
            padded_forecast = np.full(len(df_all_forecasts), np.nan)
            padded_forecast[:len(forecast_values)] = forecast_values
            df_all_forecasts[model_name] = padded_forecast
            print(f"Предупреждение: Прогноз для {model_name} был короче тестового периода и был дополнен NaN.")
    else:
        print(f"Предупреждение: Прогноз для модели '{model_name}' не найден в recursive_forecasts. Пропускаем.")

# 5. Определяем имя файла
#    Используем train_do (дата начала тестового периода) для уникальности имени
#    Предполагаем, что train_do существует и содержит строку типа 'YYYY-MM-DD'
if 'train_do' not in locals() or not isinstance(train_do, str):
    print("Предупреждение: Переменная train_do (дата начала теста) не найдена. Используется стандартное имя файла.")
    file_name_excel = "Прогнозы_моделей.xlsx"
else:
    try:
        # Извлекаем месяц и год из train_do
        # Это предполагает, что train_do в формате 'YYYY-MM-DD'
        year_month_str = pd.to_datetime(train_do).strftime('%B_%Y') # Например, 'Февраль_2023'
        file_name_excel = f"Прогнозы_{year_month_str}.xlsx"
    except Exception as e:
        print(f"Ошибка при форматировании имени файла из train_do: {e}. Используется стандартное имя.")
        file_name_excel = "Прогнозы_моделей.xlsx"


# 6. Сохраняем DataFrame в Excel
try:
    df_all_forecasts.to_excel(file_name_excel)
    print(f"\nПрогнозы успешно сохранены в файл: {file_name_excel}")
    # Выведем первые несколько строк сохраненного DataFrame для проверки
    print("\nПервые 5 строк сохраненных данных:")
    print(df_all_forecasts.head())
except Exception as e:
    print(f"Ошибка при сохранении файла Excel: {e}")